import time
plc=1
time.sleep(1500*plc)         #run after 1 houre = 3600 seconds

In [1]:
import os
import glob
import scipy
import random
import bisect
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from preprocess import *                         #ماژول استخراج پنجره ها
from data import *                          #ماژول محلی ورود داده ها
from augment import *                            #ماژول های داده افزایی
from lstm_cnn import *                           #ماژول های داده افزایی با lstm_cnn
import augment
import importlib
importlib.reload(augment)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,recall_score,precision_score
from tensorflow.keras import callbacks
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,Conv1D,Dropout,MaxPooling1D,MaxPooling2D,Flatten,Dense
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
cwd = os.getcwd() #
fullscrn()

In [2]:
def maxindx(ar):
    ar_cpy=np.array(ar)
    pred=np.argmax(ar,axis=1)
    for i in range (len(pred)):
        if pred[i]==0:
            ar_cpy[i,0]=-1000
    pred=np.argmax(ar_cpy,axis=1)
    return(pred)

<h1><center><div style="direction:rtl;font-family:B Nazanin">Importing Data</div></center></h1>

In [3]:
cls_num=12
btch=30
for i in range (1,cls_num+1):
    vars()['ecgH'+str(i)],vars()['ecg_tstH'+str(i)]=EOGHorizontalSignal(i)
os.chdir(cwd)
clses_lens=np.array([])
clses_lensH=np.array([])
i=0                               #جمع آوری داده ها و چاپ تعداد نمونه ی هر کلاس
ecgH=np.array(ecgH1)
print(1,len(vars()['ecgH'+str(1)]), end='\t')
clses_lensH=np.append(clses_lensH,len(vars()['ecgH'+str(1)]))
for i in range (2,cls_num+1):
    ecgH=np.concatenate((ecgH,vars()['ecgH'+str(i)]),axis=0)
    clses_lensH=np.append(clses_lensH,len(vars()['ecgH'+str(i)]))
    print( i,len(vars()['ecgH'+str(i)]), end='\t')

mx_sigH=max(clses_lensH)
print('\nshape ecgH=',np.shape(ecgH))

C:\ProgramData\Anaconda3\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


first_train_shape= (362, 1251)
first_test_shape= (362, 1251)
classes_quantity= 12
tr_lbls=	 {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}
Count_labels= [30 30 30 30 30 31 31 30 30 30 30 30]
max(train_feature_Altitude)= 446.32
min(train_feature_Altitude)= -1110.8
first_train_sample=
 [   1.         0.47769    0.60965 ... -110.76    -111.03    -110.9    ]
1 30	2 30	3 30	4 30	5 30	6 31	7 31	8 30	9 30	10 30	11 30	12 30	
shape ecgH= (362, 481)


In [4]:
#Vertical
for i in range (1,cls_num+1):
    vars()['ecgV'+str(i)],vars()['ecg_tstV'+str(i)]=EOGVerticalSignal(i)
os.chdir(cwd)
clses_lensV=np.array([])
i=0                               #جمع آوری داده ها و چاپ تعداد نمونه ی هر کلاس
ecgV=np.array(ecgV1)
print(1,len(vars()['ecgV'+str(1)]), end='\t')
clses_lens=np.append(clses_lens,len(vars()['ecgV'+str(1)]))
for i in range (2,cls_num+1):
    ecgV=np.concatenate((ecgV,vars()['ecgV'+str(i)]),axis=0)
    clses_lensV=np.append(clses_lensV,len(vars()['ecgV'+str(i)]))
    print( i,len(vars()['ecgV'+str(i)]), end='\t')

mx_sigV=max(clses_lensV)
print('\nshape ecgV=',np.shape(ecgV))

first_train_shape= (362, 1251)
first_test_shape= (362, 1251)
classes_quantity= 12
tr_lbls=	 {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}
Count_labels= [30 30 30 30 30 31 31 30 30 30 30 30]
max(train_feature_Altitude)= 2540.7
min(train_feature_Altitude)= -923.79
first_train_sample=
 [   1.       -5.7032   -5.8955 ... -113.16   -112.85   -112.94  ]
1 30	2 30	3 30	4 30	5 30	6 31	7 31	8 30	9 30	10 30	11 30	12 30	
shape ecgV= (362, 481)


In [5]:
for i in range (1,cls_num+1):
    #vars()['ecgHV'+str(i)]=np.array([])
    vars()['ecg'+str(i)]=np.array(np.concatenate((vars()['ecgH'+str(i)][:,0:-1],vars()['ecgV'+str(i)]),axis=1))
    vars()['ecg_tst'+str(i)]=np.array(np.concatenate((vars()['ecg_tstH'+str(i)][:,0:-1],vars()['ecg_tstV'+str(i)]),axis=1))
ecg=np.array(np.concatenate((ecgH[:,:-1],ecgV),axis=1))

In [6]:
np.shape(ecg)

(362, 961)

<h2><center><div style="direction:rtl;font-family:B Nazanin">Base Train windows</div></center></h1>

In [7]:
smpl_rte=960                                              # در ماژول ها نیز همین مقدار ثبت شده
i=0
windws=np.array([])

for cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws'+str(cls)]=np.array(vars()['ecg'+str(cls)])  
    
print("each class and its windows =")

cls_wndws=np.array([])
for cls in range (1,cls_num+1):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')

each class and its windows =
1 30	2 30	3 30	4 30	5 30	6 31	7 31	8 30	9 30	10 30	11 30	12 30	

<h1><center><div style="direction:rtl;font-family:B Nazanin">Data Augmentation</div></center></h1>

In [8]:
for alpha in range(1,7,2):
    if alpha==5:
        alpha=10
    print(alpha)


1
3
10


In [9]:
mx_wndws=int(np.max(cls_wndws)+1)   # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max =', mx_wndws)
aug_amnt_Ttl=np.empty((0,7), int) 
rpt_cnn_clsfctn=5                 #تعداد اجرای شبکه عصبی برای میانگین گیری هر حالت 
rslts=np.array([])
alpha=1                           #چند برابر کردن داده افزایی هر کلاس متناسب ضعیف بودن آن کلاس
for beta in range(1,4):
    if beta==3:
        beta=beta*2
    for X in range(1,4):          #داده افزایی تا چند برابر کلاس اکثریت
        mx_wndws=int(np.max(cls_wndws))   # حداکثر تعداد پنجره ی موجود بین کلاس ها
        mx_wndws=int(X*mx_wndws)          #داده افزایی تا چند برابر کلاس اکثریت
        scors0=np.array([64,78,55,70,94,85,84,85,78,88,63,90])         #Class Recalls without Augmentation
        AgScr=1+alpha*(100-scors0)/100                                 #Class Specific
        f_scr=np.array([83,88,87,83,85,85,85,85])                      #UnAg,Scl.2,Mag.05,Tm.2,Gs1,frqnc2,GAN,LSTMexpgdo
        f_scr_difrnc=f_scr-f_scr[0]
        for z in range (len(f_scr_difrnc)):
            if f_scr_difrnc[z]<0:
                f_scr_difrnc[z]=0
        f_scr_difrnc=f_scr_difrnc**beta
        f_scr_rtio=f_scr_difrnc/(sum(f_scr_difrnc))
        aug_amnt=np.empty((0,len(f_scr_rtio)-1), int) 
        for i in range (1,cls_num+1):
            aug_amnt=np.append(aug_amnt,[f_scr_rtio[1:]*(mx_wndws*AgScr[i-1]-cls_wndws[i-1])],axis=0)
            #vlum_win=mx_wndws-cls_wndws[i-1]             #میزان داده افزایی در روش های عمیق (پنجره)
        aug_amnt=np.int16(np.round(aug_amnt))
        
        aug_amnt_Ttl=np.append(aug_amnt_Ttl,aug_amnt,axis=0)
        #print(np.int16(100*f_scr_rtio))
        #print(aug_amnt)
        rprt=np.empty((0,3),float)
        print(aug_amnt)
        for repeat in range(1,rpt_cnn_clsfctn+1):
            xtrain=np.empty((0,smpl_rte+1), float) 
            for i in range (1,cls_num+1):                                     #تعریف آرایه ی پنجره های کلاس ها
                xtrain=np.append(xtrain,vars()['wndws'+str(i)],axis=0)
                if aug_amnt[i-1,0]>0:
                    methd=10
                    sort=-3  #-5=Dis_sam_Cls=qlty1 #-4=var_Dis=qlty2 #-3=DisSm/DisOthr=qlty3  #-2=invrs_GDO_papr=qlty4   #-1=GDO_papr=qlty5
                    dp_slct_dstrb=4            #1-sequencial 2-linear 3-beta 4-exponential 5-unfrm
                    Hrzntl=srtd_dataH(methd,i,aug_amnt[i-1,0],dp_slct_dstrb,sort)
                    Vrtcl=srtd_dataV(methd,i,aug_amnt[i-1,0],dp_slct_dstrb,sort)
                    HrzntlVrtcl=np.concatenate((Hrzntl[:,:-1],Vrtcl),axis=1)
                    xtrain=np.append(xtrain,HrzntlVrtcl,axis=0)
                if aug_amnt[i-1,1]>0:
                    methd=20
                    sort=-3
                    dp_slct_dstrb=2
                    Hrzntl=srtd_dataH(methd,i,aug_amnt[i-1,1],dp_slct_dstrb,sort)
                    Vrtcl=srtd_dataV(methd,i,aug_amnt[i-1,1],dp_slct_dstrb,sort)
                    HrzntlVrtcl=np.concatenate((Hrzntl[:,:-1],Vrtcl),axis=1)
                    xtrain=np.append(xtrain,HrzntlVrtcl,axis=0)
                if aug_amnt[i-1,2]>0:
                    methd=30
                    sort=-3
                    dp_slct_dstrb=4
                    Hrzntl=srtd_dataH(methd,i,aug_amnt[i-1,2],dp_slct_dstrb,sort)
                    Vrtcl=srtd_dataV(methd,i,aug_amnt[i-1,2],dp_slct_dstrb,sort)
                    HrzntlVrtcl=np.concatenate((Hrzntl[:,:-1],Vrtcl),axis=1)
                    xtrain=np.append(xtrain,HrzntlVrtcl,axis=0)
                if aug_amnt[i-1,3]>0:
                    methd=40
                    sort=-1
                    dp_slct_dstrb=4
                    Hrzntl=srtd_dataH(methd,i,aug_amnt[i-1,3],dp_slct_dstrb,sort)
                    Vrtcl=srtd_dataV(methd,i,aug_amnt[i-1,3],dp_slct_dstrb,sort)
                    HrzntlVrtcl=np.concatenate((Hrzntl[:,:-1],Vrtcl),axis=1)
                    xtrain=np.append(xtrain,HrzntlVrtcl,axis=0)
                if aug_amnt[i-1,4]>0:
                    methd=50
                    sort=-3
                    dp_slct_dstrb=4
                    Hrzntl=srtd_dataH(methd,i,aug_amnt[i-1,4],dp_slct_dstrb,sort)
                    Vrtcl=srtd_dataV(methd,i,aug_amnt[i-1,4],dp_slct_dstrb,sort)
                    HrzntlVrtcl=np.concatenate((Hrzntl[:,:-1],Vrtcl),axis=1)
                    xtrain=np.append(xtrain,HrzntlVrtcl,axis=0)
                if aug_amnt[i-1,5]>0:
                    methd=60
                    sort=-1
                    dp_slct_dstrb=4
                    Hrzntl=srtd_dataH(methd,i,aug_amnt[i-1,5],dp_slct_dstrb,sort)
                    Vrtcl=srtd_dataV(methd,i,aug_amnt[i-1,5],dp_slct_dstrb,sort)
                    HrzntlVrtcl=np.concatenate((Hrzntl[:,:-1],Vrtcl),axis=1)
                    xtrain=np.append(xtrain,HrzntlVrtcl,axis=0)
                if aug_amnt[i-1,6]>0:
                    methd=70
                    sort=-1
                    dp_slct_dstrb=2
                    Hrzntl=srtd_dataH(methd,i,aug_amnt[i-1,6],dp_slct_dstrb,sort)
                    Vrtcl=srtd_dataV(methd,i,aug_amnt[i-1,6],dp_slct_dstrb,sort)
                    HrzntlVrtcl=np.concatenate((Hrzntl[:,:-1],Vrtcl),axis=1)
                    xtrain=np.append(xtrain,HrzntlVrtcl,axis=0)
                #print('class ', i, 'train shape = ', np.shape(xtrain))

            # Normalization train windows
            mx_aug=np.max(xtrain[:,:-1])
            mx=np.max(ecg[:,:-1])
            mn_aug=np.min(xtrain[:,:-1])
            mn=np.min(ecg[:,:-1])
            xtrain[:,:-1]= 2*(xtrain[:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1


            ###Test Windows:
            wndws_test=np.empty((0,len(wndws1[0])), float)
            i=0
            windws=np.array([])
            for cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های تست
                vars()['wndws_tst'+str(cls)]=np.empty((0,len(wndws1[0])), float)
                vars()['wndws_tst'+str(cls)]=np.append(vars()['wndws_tst'+str(cls)],vars()['ecg_tst'+str(cls)],axis=0)   

            # Normalization test windows
            for i in range (1,cls_num+1):                
                cls=i
                vars()['wndws_tst'+str(i)][:,:-1]= 2*(vars()['wndws_tst'+str(i)][:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1
            for cls in range (1,cls_num+1):                                # آرایه ی تعداد پنجره ی هر کلاس
                wns=len(vars()['wndws_tst'+str(cls)])
                wndws_test=np.append(wndws_test,vars()['wndws_tst'+str(cls)],axis=0)

            Xtest=wndws_test[:,:-1]
            ytest=np.int16(wndws_test[:,-1])

            trainx=np.random.permutation(xtrain)
            Xtrain=np.array(trainx[:,:-1])
            ytrain=np.int16(trainx[:,-1])

            X_train=np.array(Xtrain)
            y_train=np.array(ytrain)
            X_test=np.array(Xtest)
            y_test=np.array(ytest)
            X_valid=np.array(Xtrain)
            y_valid=np.array(ytrain)
            #print('X_train => ', X_train.shape)
            #print('y_train => ', y_train.shape)
            #print('X_test  => ', X_test.shape)
            #print('y_test  => ', y_test.shape)
            #print('X_valid  => ', X_valid.shape)
            #print('y_valid  => ', y_valid.shape)

            y_train_cat=to_categorical(y_train)
            y_valid_cat=to_categorical(y_valid)
            y_test_cat=to_categorical(y_test)
            X_train = np.expand_dims(X_train, axis=2)
            X_valid = np.expand_dims(X_valid, axis=2)
            X_test = np.expand_dims(X_test, axis=2)

            ####### NETWORK #######
            accuracy=0
            acc_crs=np.array([])
            ##for i in range (10):
            ##    if accuracy<0.2 :
            verbose, epochs, batch_size = 0, 1000, btch
            n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
            steps_per_epoch = len(X_train)//batch_size
            validation_steps = len(X_valid)//batch_size # if you have test data
            model_crs = Sequential()
            BatchNormalization()
            model_crs.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
            #model_crs.add(MaxPooling1D(pool_size=3))
            BatchNormalization()
            model_crs.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
            #model_crs.add(Dropout(0.5))
            model_crs.add(MaxPooling1D(pool_size=3))
            BatchNormalization()
            model_crs.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
            model_crs.add(MaxPooling1D(pool_size=3))
            BatchNormalization()
            model_crs.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
            model_crs.add(MaxPooling1D(pool_size=3))
            BatchNormalization()
            model_crs.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
            model_crs.add(MaxPooling1D(pool_size=3))
            '''model_crs.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
            model_crs.add(MaxPooling1D(pool_size=3))
            model_crs.add(Conv1D(filters=1024, kernel_size=3, activation='relu'))
            model_crs.add(MaxPooling1D(pool_size=3))
            '''
            model_crs.add(Flatten())
            #model_crs.add(Dropout(0.5))
            model_crs.add(Dense(2000, activation='relu'))
            model_crs.add(Dense(n_outputs, activation='softmax'))
            model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
            # fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy
            earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 20, restore_best_weights = True)
            model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])
            # evaluate model_crs
            _, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=verbose)
            predict_x=model_crs.predict(X_test)              # Function 1
            y_pred_crs=maxindx(predict_x)                    # function from augment.py to remove 0 index predictions
            rprt0=classification_report(y_test, y_pred_crs,output_dict=True)
            rprt_row=np.array([])
            rprt_row=np.append(rprt_row,rprt0['accuracy'])
            rprt_row=np.append(rprt_row,rprt0['macro avg']['f1-score'])
            rprt_row=np.append(rprt_row,rprt0['weighted avg']['f1-score'])
            rprt=np.append(rprt,[rprt_row],axis=0)
        #print('rprt=',rprt)
        maxm=rprt[np.argmax(rprt[:,1])]
        #print('maxm=',maxm)
        avrg=np.mean(rprt,axis=0)
        #print('avrg=',avrg)
        rslts=np.append(rslts,['Beta '+str(beta)+'  X '+str(X)+'  average= '+str(np.int16(100*avrg))+'  max= '+str(np.int16(100*maxm))+'  var= '+str(np.int16(np.var(100*rprt,axis=0)))],axis=0)
        #print('rslts=',rslts)


 max = 32
[[4 3 0 1 1 1 1]
 [2 2 0 1 1 1 1]
 [4 4 0 2 2 2 2]
 [3 2 0 1 1 1 1]
 [1 1 0 0 0 0 0]
 [1 1 0 1 1 1 1]
 [1 1 0 1 1 1 1]
 [2 1 0 1 1 1 1]
 [2 2 0 1 1 1 1]
 [1 1 0 1 1 1 1]
 [4 3 0 1 1 1 1]
 [1 1 0 0 0 0 0]]
12/12 [==============================] - 0s 4ms/step
[[16 13  0  6  6  6  6]
 [13 11  0  5  5  5  5]
 [18 14  0  7  7  7  7]
 [15 12  0  6  6  6  6]
 [11  8  0  4  4  4  4]
 [12  9  0  5  5  5  5]
 [12 10  0  5  5  5  5]
 [12 10  0  5  5  5  5]
 [13 11  0  5  5  5  5]
 [12  9  0  5  5  5  5]
 [16 13  0  6  6  6  6]
 [11  9  0  4  4  4  4]]
12/12 [==============================] - 0s 3ms/step
[[28 23  0 11 11 11 11]
 [25 20  0 10 10 10 10]
 [31 25  0 12 12 12 12]
 [27 21  0 11 11 11 11]
 [20 16  0  8  8  8  8]
 [22 18  0  9  9  9  9]
 [23 18  0  9  9  9  9]
 [23 18  0  9  9  9  9]
 [25 20  0 10 10 10 10]
 [22 17  0  9  9  9  9]
 [29 23  0 11 11 11 11]
 [21 17  0  9  9  9  9]]
12/12 [==============================] - 0s 3ms/step
[[5 3 0 1 1 1 1]
 [3 2 0 1 1 1 1]
 [7 4 0 1 1 1

In [10]:
aug_amnt_Ttl

array([[ 4,  3,  0,  1,  1,  1,  1],
       [ 2,  2,  0,  1,  1,  1,  1],
       [ 4,  4,  0,  2,  2,  2,  2],
       [ 3,  2,  0,  1,  1,  1,  1],
       [ 1,  1,  0,  0,  0,  0,  0],
       [ 1,  1,  0,  1,  1,  1,  1],
       [ 1,  1,  0,  1,  1,  1,  1],
       [ 2,  1,  0,  1,  1,  1,  1],
       [ 2,  2,  0,  1,  1,  1,  1],
       [ 1,  1,  0,  1,  1,  1,  1],
       [ 4,  3,  0,  1,  1,  1,  1],
       [ 1,  1,  0,  0,  0,  0,  0],
       [16, 13,  0,  6,  6,  6,  6],
       [13, 11,  0,  5,  5,  5,  5],
       [18, 14,  0,  7,  7,  7,  7],
       [15, 12,  0,  6,  6,  6,  6],
       [11,  8,  0,  4,  4,  4,  4],
       [12,  9,  0,  5,  5,  5,  5],
       [12, 10,  0,  5,  5,  5,  5],
       [12, 10,  0,  5,  5,  5,  5],
       [13, 11,  0,  5,  5,  5,  5],
       [12,  9,  0,  5,  5,  5,  5],
       [16, 13,  0,  6,  6,  6,  6],
       [11,  9,  0,  4,  4,  4,  4],
       [28, 23,  0, 11, 11, 11, 11],
       [25, 20,  0, 10, 10, 10, 10],
       [31, 25,  0, 12, 12, 12, 12],
 

In [11]:
rslts

array(['Beta 1  X 1  average= [86 86 85]  max= [87 87 87]  var= [2 2 2]',
       'Beta 1  X 2  average= [86 86 86]  max= [87 87 87]  var= [0 1 1]',
       'Beta 1  X 3  average= [86 86 86]  max= [88 88 88]  var= [4 4 4]',
       'Beta 2  X 1  average= [84 84 84]  max= [85 85 85]  var= [1 1 1]',
       'Beta 2  X 2  average= [85 85 85]  max= [87 87 87]  var= [1 1 1]',
       'Beta 2  X 3  average= [88 88 88]  max= [90 89 89]  var= [1 1 1]',
       'Beta 6  X 1  average= [82 82 82]  max= [83 82 82]  var= [0 0 0]',
       'Beta 6  X 2  average= [85 85 85]  max= [88 87 87]  var= [1 1 1]',
       'Beta 6  X 3  average= [85 85 85]  max= [88 87 87]  var= [3 3 3]'],
      dtype='<U63')

In [14]:
results1=rslts

In [15]:
mx_wndws=int(np.max(cls_wndws)+1)   # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max =', mx_wndws)
aug_amnt_Ttl=np.empty((0,7), int) 
rpt_cnn_clsfctn=5                 #تعداد اجرای شبکه عصبی برای میانگین گیری هر حالت 
rslts=np.array([])
alpha=1                           #چند برابر کردن داده افزایی هر کلاس متناسب ضعیف بودن آن کلاس
for beta in range(1,4):
    if beta==3:
        beta=beta*2
    for X in range(1,4):          #داده افزایی تا چند برابر کلاس اکثریت
        mx_wndws=int(np.max(cls_wndws))   # حداکثر تعداد پنجره ی موجود بین کلاس ها
        mx_wndws=int(X*mx_wndws)          #داده افزایی تا چند برابر کلاس اکثریت
        scors0=np.array([64,78,55,70,94,85,84,85,78,88,63,90])         #Class Recalls without Augmentation
        AgScr=1+alpha*((100-scors0)/(np.max(100-scors0)))**beta            #Class Specific
        f_scr=np.array([83,88,87,83,85,85,85,85])                      #UnAg,Scl.2,Mag.05,Tm.2,Gs1,frqnc2,GAN,LSTMexpgdo
        f_scr_difrnc=f_scr-f_scr[0]
        for z in range (len(f_scr_difrnc)):
            if f_scr_difrnc[z]<0:
                f_scr_difrnc[z]=0
        f_scr_difrnc=f_scr_difrnc**beta
        f_scr_rtio=f_scr_difrnc/(sum(f_scr_difrnc))
        aug_amnt=np.empty((0,len(f_scr_rtio)-1), int) 
        for i in range (1,cls_num+1):
            aug_amnt=np.append(aug_amnt,[f_scr_rtio[1:]*(mx_wndws*AgScr[i-1]-cls_wndws[i-1])],axis=0)
            #vlum_win=mx_wndws-cls_wndws[i-1]             #میزان داده افزایی در روش های عمیق (پنجره)
        aug_amnt=np.int16(np.round(aug_amnt))
        
        aug_amnt_Ttl=np.append(aug_amnt_Ttl,aug_amnt,axis=0)
        #print(np.int16(100*f_scr_rtio))
        #print(aug_amnt)
        rprt=np.empty((0,3),float)
        print(aug_amnt)
        for repeat in range(1,rpt_cnn_clsfctn+1):
            xtrain=np.empty((0,smpl_rte+1), float) 
            for i in range (1,cls_num+1):                                     #تعریف آرایه ی پنجره های کلاس ها
                xtrain=np.append(xtrain,vars()['wndws'+str(i)],axis=0)
                if aug_amnt[i-1,0]>0:
                    methd=10
                    sort=-3  #-5=Dis_sam_Cls=qlty1 #-4=var_Dis=qlty2 #-3=DisSm/DisOthr=qlty3  #-2=invrs_GDO_papr=qlty4   #-1=GDO_papr=qlty5
                    dp_slct_dstrb=4            #1-sequencial 2-linear 3-beta 4-exponential 5-unfrm
                    Hrzntl=srtd_dataH(methd,i,aug_amnt[i-1,0],dp_slct_dstrb,sort)
                    Vrtcl=srtd_dataV(methd,i,aug_amnt[i-1,0],dp_slct_dstrb,sort)
                    HrzntlVrtcl=np.concatenate((Hrzntl[:,:-1],Vrtcl),axis=1)
                    xtrain=np.append(xtrain,HrzntlVrtcl,axis=0)
                if aug_amnt[i-1,1]>0:
                    methd=20
                    sort=-3
                    dp_slct_dstrb=2
                    Hrzntl=srtd_dataH(methd,i,aug_amnt[i-1,1],dp_slct_dstrb,sort)
                    Vrtcl=srtd_dataV(methd,i,aug_amnt[i-1,1],dp_slct_dstrb,sort)
                    HrzntlVrtcl=np.concatenate((Hrzntl[:,:-1],Vrtcl),axis=1)
                    xtrain=np.append(xtrain,HrzntlVrtcl,axis=0)
                if aug_amnt[i-1,2]>0:
                    methd=30
                    sort=-3
                    dp_slct_dstrb=4
                    Hrzntl=srtd_dataH(methd,i,aug_amnt[i-1,2],dp_slct_dstrb,sort)
                    Vrtcl=srtd_dataV(methd,i,aug_amnt[i-1,2],dp_slct_dstrb,sort)
                    HrzntlVrtcl=np.concatenate((Hrzntl[:,:-1],Vrtcl),axis=1)
                    xtrain=np.append(xtrain,HrzntlVrtcl,axis=0)
                if aug_amnt[i-1,3]>0:
                    methd=40
                    sort=-1
                    dp_slct_dstrb=4
                    Hrzntl=srtd_dataH(methd,i,aug_amnt[i-1,3],dp_slct_dstrb,sort)
                    Vrtcl=srtd_dataV(methd,i,aug_amnt[i-1,3],dp_slct_dstrb,sort)
                    HrzntlVrtcl=np.concatenate((Hrzntl[:,:-1],Vrtcl),axis=1)
                    xtrain=np.append(xtrain,HrzntlVrtcl,axis=0)
                if aug_amnt[i-1,4]>0:
                    methd=50
                    sort=-3
                    dp_slct_dstrb=4
                    Hrzntl=srtd_dataH(methd,i,aug_amnt[i-1,4],dp_slct_dstrb,sort)
                    Vrtcl=srtd_dataV(methd,i,aug_amnt[i-1,4],dp_slct_dstrb,sort)
                    HrzntlVrtcl=np.concatenate((Hrzntl[:,:-1],Vrtcl),axis=1)
                    xtrain=np.append(xtrain,HrzntlVrtcl,axis=0)
                if aug_amnt[i-1,5]>0:
                    methd=60
                    sort=-1
                    dp_slct_dstrb=4
                    Hrzntl=srtd_dataH(methd,i,aug_amnt[i-1,5],dp_slct_dstrb,sort)
                    Vrtcl=srtd_dataV(methd,i,aug_amnt[i-1,5],dp_slct_dstrb,sort)
                    HrzntlVrtcl=np.concatenate((Hrzntl[:,:-1],Vrtcl),axis=1)
                    xtrain=np.append(xtrain,HrzntlVrtcl,axis=0)
                if aug_amnt[i-1,6]>0:
                    methd=70
                    sort=-1
                    dp_slct_dstrb=2
                    Hrzntl=srtd_dataH(methd,i,aug_amnt[i-1,6],dp_slct_dstrb,sort)
                    Vrtcl=srtd_dataV(methd,i,aug_amnt[i-1,6],dp_slct_dstrb,sort)
                    HrzntlVrtcl=np.concatenate((Hrzntl[:,:-1],Vrtcl),axis=1)
                    xtrain=np.append(xtrain,HrzntlVrtcl,axis=0)
                #print('class ', i, 'train shape = ', np.shape(xtrain))

            # Normalization train windows
            mx_aug=np.max(xtrain[:,:-1])
            mx=np.max(ecg[:,:-1])
            mn_aug=np.min(xtrain[:,:-1])
            mn=np.min(ecg[:,:-1])
            xtrain[:,:-1]= 2*(xtrain[:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1


            ###Test Windows:
            wndws_test=np.empty((0,len(wndws1[0])), float)
            i=0
            windws=np.array([])
            for cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های تست
                vars()['wndws_tst'+str(cls)]=np.empty((0,len(wndws1[0])), float)
                vars()['wndws_tst'+str(cls)]=np.append(vars()['wndws_tst'+str(cls)],vars()['ecg_tst'+str(cls)],axis=0)   

            # Normalization test windows
            for i in range (1,cls_num+1):                
                cls=i
                vars()['wndws_tst'+str(i)][:,:-1]= 2*(vars()['wndws_tst'+str(i)][:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1
            for cls in range (1,cls_num+1):                                # آرایه ی تعداد پنجره ی هر کلاس
                wns=len(vars()['wndws_tst'+str(cls)])
                wndws_test=np.append(wndws_test,vars()['wndws_tst'+str(cls)],axis=0)

            Xtest=wndws_test[:,:-1]
            ytest=np.int16(wndws_test[:,-1])

            trainx=np.random.permutation(xtrain)
            Xtrain=np.array(trainx[:,:-1])
            ytrain=np.int16(trainx[:,-1])

            X_train=np.array(Xtrain)
            y_train=np.array(ytrain)
            X_test=np.array(Xtest)
            y_test=np.array(ytest)
            X_valid=np.array(Xtrain)
            y_valid=np.array(ytrain)
            #print('X_train => ', X_train.shape)
            #print('y_train => ', y_train.shape)
            #print('X_test  => ', X_test.shape)
            #print('y_test  => ', y_test.shape)
            #print('X_valid  => ', X_valid.shape)
            #print('y_valid  => ', y_valid.shape)

            y_train_cat=to_categorical(y_train)
            y_valid_cat=to_categorical(y_valid)
            y_test_cat=to_categorical(y_test)
            X_train = np.expand_dims(X_train, axis=2)
            X_valid = np.expand_dims(X_valid, axis=2)
            X_test = np.expand_dims(X_test, axis=2)

            ####### NETWORK #######
            accuracy=0
            acc_crs=np.array([])
            ##for i in range (10):
            ##    if accuracy<0.2 :
            verbose, epochs, batch_size = 0, 1000, btch
            n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
            steps_per_epoch = len(X_train)//batch_size
            validation_steps = len(X_valid)//batch_size # if you have test data
            model_crs = Sequential()
            BatchNormalization()
            model_crs.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
            #model_crs.add(MaxPooling1D(pool_size=3))
            BatchNormalization()
            model_crs.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
            #model_crs.add(Dropout(0.5))
            model_crs.add(MaxPooling1D(pool_size=3))
            BatchNormalization()
            model_crs.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
            model_crs.add(MaxPooling1D(pool_size=3))
            BatchNormalization()
            model_crs.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
            model_crs.add(MaxPooling1D(pool_size=3))
            BatchNormalization()
            model_crs.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
            model_crs.add(MaxPooling1D(pool_size=3))
            '''model_crs.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
            model_crs.add(MaxPooling1D(pool_size=3))
            model_crs.add(Conv1D(filters=1024, kernel_size=3, activation='relu'))
            model_crs.add(MaxPooling1D(pool_size=3))
            '''
            model_crs.add(Flatten())
            #model_crs.add(Dropout(0.5))
            model_crs.add(Dense(2000, activation='relu'))
            model_crs.add(Dense(n_outputs, activation='softmax'))
            model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
            # fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy
            earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 20, restore_best_weights = True)
            model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])
            # evaluate model_crs
            _, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=verbose)
            predict_x=model_crs.predict(X_test)              # Function 1
            y_pred_crs=maxindx(predict_x)                    # function from augment.py to remove 0 index predictions
            rprt0=classification_report(y_test, y_pred_crs,output_dict=True)
            rprt_row=np.array([])
            rprt_row=np.append(rprt_row,rprt0['accuracy'])
            rprt_row=np.append(rprt_row,rprt0['macro avg']['f1-score'])
            rprt_row=np.append(rprt_row,rprt0['weighted avg']['f1-score'])
            rprt=np.append(rprt,[rprt_row],axis=0)
        #print('rprt=',rprt)
        maxm=rprt[np.argmax(rprt[:,1])]
        #print('maxm=',maxm)
        avrg=np.mean(rprt,axis=0)
        #print('avrg=',avrg)
        rslts=np.append(rslts,['Beta '+str(beta)+'  X '+str(X)+'  average= '+str(np.int16(100*avrg))+'  max= '+str(np.int16(100*maxm))+'  var= '+str(np.int16(np.var(100*rprt,axis=0)))],axis=0)
        #print('rslts=',rslts)


 max = 32
[[8 6 0 3 3 3 3]
 [5 4 0 2 2 2 2]
 [9 8 0 4 4 4 4]
 [6 5 0 3 3 3 3]
 [2 1 0 1 1 1 1]
 [3 2 0 1 1 1 1]
 [3 3 0 1 1 1 1]
 [3 3 0 1 1 1 1]
 [5 4 0 2 2 2 2]
 [3 2 0 1 1 1 1]
 [8 6 0 3 3 3 3]
 [2 2 0 1 1 1 1]]
12/12 [==============================] - 0s 3ms/step
[[24 19  0 10 10 10 10]
 [18 15  0  7  7  7  7]
 [28 22  0 11 11 11 11]
 [22 17  0  9  9  9  9]
 [12  9  0  5  5  5  5]
 [15 12  0  6  6  6  6]
 [16 12  0  6  6  6  6]
 [15 12  0  6  6  6  6]
 [18 15  0  7  7  7  7]
 [14 11  0  6  6  6  6]
 [24 20  0 10 10 10 10]
 [13 11  0  5  5  5  5]]
12/12 [==============================] - 0s 3ms/step
[[40 32  0 16 16 16 16]
 [32 26  0 13 13 13 13]
 [46 37  0 18 18 18 18]
 [37 29  0 15 15 15 15]
 [22 18  0  9  9  9  9]
 [27 22  0 11 11 11 11]
 [28 22  0 11 11 11 11]
 [28 22  0 11 11 11 11]
 [32 26  0 13 13 13 13]
 [26 21  0 10 10 10 10]
 [41 33  0 16 16 16 16]
 [25 20  0 10 10 10 10]]
12/12 [==============================] - 0s 3ms/step
[[ 9  6  0  1  1  1  1]
 [ 4  2  0  1  1  1  1]

In [16]:
aug_amnt_Ttl

array([[  8,   6,   0,   3,   3,   3,   3],
       [  5,   4,   0,   2,   2,   2,   2],
       [  9,   8,   0,   4,   4,   4,   4],
       [  6,   5,   0,   3,   3,   3,   3],
       [  2,   1,   0,   1,   1,   1,   1],
       [  3,   2,   0,   1,   1,   1,   1],
       [  3,   3,   0,   1,   1,   1,   1],
       [  3,   3,   0,   1,   1,   1,   1],
       [  5,   4,   0,   2,   2,   2,   2],
       [  3,   2,   0,   1,   1,   1,   1],
       [  8,   6,   0,   3,   3,   3,   3],
       [  2,   2,   0,   1,   1,   1,   1],
       [ 24,  19,   0,  10,  10,  10,  10],
       [ 18,  15,   0,   7,   7,   7,   7],
       [ 28,  22,   0,  11,  11,  11,  11],
       [ 22,  17,   0,   9,   9,   9,   9],
       [ 12,   9,   0,   5,   5,   5,   5],
       [ 15,  12,   0,   6,   6,   6,   6],
       [ 16,  12,   0,   6,   6,   6,   6],
       [ 15,  12,   0,   6,   6,   6,   6],
       [ 18,  15,   0,   7,   7,   7,   7],
       [ 14,  11,   0,   6,   6,   6,   6],
       [ 24,  20,   0,  10,  10,

In [17]:
rslts

array(['Beta 1  X 1  average= [84 84 84]  max= [85 85 85]  var= [1 1 1]',
       'Beta 1  X 2  average= [88 87 87]  max= [88 88 88]  var= [0 0 0]',
       'Beta 1  X 3  average= [87 87 87]  max= [88 88 88]  var= [1 1 1]',
       'Beta 2  X 1  average= [85 85 85]  max= [88 88 88]  var= [2 2 2]',
       'Beta 2  X 2  average= [87 87 87]  max= [88 88 88]  var= [0 0 0]',
       'Beta 2  X 3  average= [88 88 88]  max= [90 90 89]  var= [0 0 0]',
       'Beta 6  X 1  average= [84 84 84]  max= [87 87 87]  var= [3 2 2]',
       'Beta 6  X 2  average= [86 85 85]  max= [87 86 86]  var= [0 0 0]',
       'Beta 6  X 3  average= [86 86 86]  max= [89 89 89]  var= [3 4 4]'],
      dtype='<U63')

In [18]:
results2=rslts

In [19]:
mx_wndws=int(np.max(cls_wndws)+1)   # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max =', mx_wndws)
aug_amnt_Ttl=np.empty((0,7), int) 
rslts=np.array([])
X=1                               #داده افزایی تا چند برابر کلاس اکثریت
for beta in range(1,4):
    if beta==3:
        beta=beta*2
    rprt=np.array([])
    for alpha in range(1,7,2):    #چند برابر کردن داده افزایی هر کلاس متناسب ضعیف بودن آن کلاس
        if alpha==5:
            alpha=10
        scors0=np.array([64,78,55,70,94,85,84,85,78,88,63,90])         #Class Recalls without Augmentation
        AgScr=1+alpha*(100-scors0)/100            #Class Specific

        f_scr=np.array([83,88,87,83,85,85,85,85])                      #UnAg,Scl.2,Mag.05,Tm.2,Gs1,frqnc2,GAN,LSTMexpgdo
        f_scr_difrnc=f_scr-f_scr[0]
        for z in range (len(f_scr_difrnc)):
            if f_scr_difrnc[z]<0:
                f_scr_difrnc[z]=0
        f_scr_difrnc=f_scr_difrnc**beta
        f_scr_rtio=f_scr_difrnc/(sum(f_scr_difrnc))
        aug_amnt=np.empty((0,len(f_scr_rtio)-1), int) 

        for i in range (1,cls_num+1):
            aug_amnt=np.append(aug_amnt,[f_scr_rtio[1:]*(mx_wndws*AgScr[i-1]-cls_wndws[i-1])],axis=0)
            #vlum_win=mx_wndws-cls_wndws[i-1]             #میزان داده افزایی در روش های عمیق (پنجره)
        aug_amnt=np.int16(np.round(aug_amnt))
        aug_amnt_Ttl=np.append(aug_amnt_Ttl,aug_amnt,axis=0)
        #print(np.int16(100*f_scr_rtio))
        #print(aug_amnt)
        rprt=np.empty((0,3),float)
        print(aug_amnt)
        for repeat in range(1,rpt_cnn_clsfctn+1):
            xtrain=np.empty((0,smpl_rte+1), float) 
            for i in range (1,cls_num+1):                                     #تعریف آرایه ی پنجره های کلاس ها
                xtrain=np.append(xtrain,vars()['wndws'+str(i)],axis=0)
                if aug_amnt[i-1,0]>0:
                    methd=10
                    sort=-3  #-5=Dis_sam_Cls=qlty1 #-4=var_Dis=qlty2 #-3=DisSm/DisOthr=qlty3  #-2=invrs_GDO_papr=qlty4   #-1=GDO_papr=qlty5
                    dp_slct_dstrb=4            #1-sequencial 2-linear 3-beta 4-exponential 5-unfrm
                    Hrzntl=srtd_dataH(methd,i,aug_amnt[i-1,0],dp_slct_dstrb,sort)
                    Vrtcl=srtd_dataV(methd,i,aug_amnt[i-1,0],dp_slct_dstrb,sort)
                    HrzntlVrtcl=np.concatenate((Hrzntl[:,:-1],Vrtcl),axis=1)
                    xtrain=np.append(xtrain,HrzntlVrtcl,axis=0)
                if aug_amnt[i-1,1]>0:
                    methd=20
                    sort=-3
                    dp_slct_dstrb=2
                    Hrzntl=srtd_dataH(methd,i,aug_amnt[i-1,0],dp_slct_dstrb,sort)
                    Vrtcl=srtd_dataV(methd,i,aug_amnt[i-1,0],dp_slct_dstrb,sort)
                    HrzntlVrtcl=np.concatenate((Hrzntl[:,:-1],Vrtcl),axis=1)
                    xtrain=np.append(xtrain,HrzntlVrtcl,axis=0)
                if aug_amnt[i-1,2]>0:
                    methd=30
                    sort=-3
                    dp_slct_dstrb=4
                    Hrzntl=srtd_dataH(methd,i,aug_amnt[i-1,2],dp_slct_dstrb,sort)
                    Vrtcl=srtd_dataV(methd,i,aug_amnt[i-1,2],dp_slct_dstrb,sort)
                    HrzntlVrtcl=np.concatenate((Hrzntl[:,:-1],Vrtcl),axis=1)
                    xtrain=np.append(xtrain,HrzntlVrtcl,axis=0)
                if aug_amnt[i-1,3]>0:
                    methd=40
                    sort=-1
                    dp_slct_dstrb=4
                    Hrzntl=srtd_dataH(methd,i,aug_amnt[i-1,3],dp_slct_dstrb,sort)
                    Vrtcl=srtd_dataV(methd,i,aug_amnt[i-1,3],dp_slct_dstrb,sort)
                    HrzntlVrtcl=np.concatenate((Hrzntl[:,:-1],Vrtcl),axis=1)
                    xtrain=np.append(xtrain,HrzntlVrtcl,axis=0)
                if aug_amnt[i-1,4]>0:
                    methd=50
                    sort=-3
                    dp_slct_dstrb=4
                    Hrzntl=srtd_dataH(methd,i,aug_amnt[i-1,4],dp_slct_dstrb,sort)
                    Vrtcl=srtd_dataV(methd,i,aug_amnt[i-1,4],dp_slct_dstrb,sort)
                    HrzntlVrtcl=np.concatenate((Hrzntl[:,:-1],Vrtcl),axis=1)
                    xtrain=np.append(xtrain,HrzntlVrtcl,axis=0)
                if aug_amnt[i-1,5]>0:
                    methd=60
                    sort=-1
                    dp_slct_dstrb=4
                    Hrzntl=srtd_dataH(methd,i,aug_amnt[i-1,5],dp_slct_dstrb,sort)
                    Vrtcl=srtd_dataV(methd,i,aug_amnt[i-1,5],dp_slct_dstrb,sort)
                    HrzntlVrtcl=np.concatenate((Hrzntl[:,:-1],Vrtcl),axis=1)
                    xtrain=np.append(xtrain,HrzntlVrtcl,axis=0)
                if aug_amnt[i-1,6]>0:
                    methd=70
                    sort=-1
                    dp_slct_dstrb=2
                    Hrzntl=srtd_dataH(methd,i,aug_amnt[i-1,6],dp_slct_dstrb,sort)
                    Vrtcl=srtd_dataV(methd,i,aug_amnt[i-1,6],dp_slct_dstrb,sort)
                    HrzntlVrtcl=np.concatenate((Hrzntl[:,:-1],Vrtcl),axis=1)
                    xtrain=np.append(xtrain,HrzntlVrtcl,axis=0)
                #print('class ', i, 'train shape = ', np.shape(xtrain))

            # Normalization train windows
            mx_aug=np.max(xtrain[:,:-1])
            mx=np.max(ecg[:,:-1])
            mn_aug=np.min(xtrain[:,:-1])
            mn=np.min(ecg[:,:-1])
            xtrain[:,:-1]= 2*(xtrain[:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1


            ###Test Windows:
            wndws_test=np.empty((0,len(wndws1[0])), float)
            i=0
            windws=np.array([])
            for cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های آموزش اصلی
                vars()['wndws_tst'+str(cls)]=np.empty((0,len(wndws1[0])), float)
                vars()['wndws_tst'+str(cls)]=np.append(vars()['wndws_tst'+str(cls)],vars()['ecg_tst'+str(cls)],axis=0)   

            # Normalization test windows
            for i in range (1,cls_num+1):                
                cls=i
                #print('\n cls', i, ' >> ')
                #print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
                #print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
                vars()['wndws_tst'+str(i)][:,:-1]= 2*(vars()['wndws_tst'+str(i)][:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1
                #print('after normalizing >>')
                #print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
                #print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
            #print("each class and its windows =")                    #تجمیع کلاس های تست
            for cls in range (1,cls_num+1):                                # آرایه ی تعداد پنجره ی هر کلاس
                wns=len(vars()['wndws_tst'+str(cls)])
                wndws_test=np.append(wndws_test,vars()['wndws_tst'+str(cls)],axis=0)
            Xtest=wndws_test[:,:-1]
            ytest=np.int16(wndws_test[:,-1])

            trainx=np.random.permutation(xtrain)
            Xtrain=np.array(trainx[:,:-1])
            ytrain=np.int16(trainx[:,-1])

            X_train=np.array(Xtrain)
            y_train=np.array(ytrain)
            X_test=np.array(Xtest)
            y_test=np.array(ytest)
            X_valid=np.array(Xtrain)
            y_valid=np.array(ytrain)
            #print('X_train => ', X_train.shape)
            #print('y_train => ', y_train.shape)
            #print('X_test  => ', X_test.shape)
            #print('y_test  => ', y_test.shape)
            #print('X_valid  => ', X_valid.shape)
            #print('y_valid  => ', y_valid.shape)

            y_train_cat=to_categorical(y_train)
            y_valid_cat=to_categorical(y_valid)
            y_test_cat=to_categorical(y_test)
            X_train = np.expand_dims(X_train, axis=2)
            X_valid = np.expand_dims(X_valid, axis=2)
            X_test = np.expand_dims(X_test, axis=2)

            ####### NETWORK #######
            accuracy=0
            acc_crs=np.array([])
            ##for i in range (10):
            ##    if accuracy<0.2 :
            verbose, epochs, batch_size = 0, 1000, btch
            n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
            steps_per_epoch = len(X_train)//batch_size
            validation_steps = len(X_valid)//batch_size # if you have test data
            model_crs = Sequential()
            BatchNormalization()
            model_crs.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
            #model_crs.add(MaxPooling1D(pool_size=3))
            BatchNormalization()
            model_crs.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
            #model_crs.add(Dropout(0.5))
            model_crs.add(MaxPooling1D(pool_size=3))
            BatchNormalization()
            model_crs.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
            model_crs.add(MaxPooling1D(pool_size=3))
            BatchNormalization()
            model_crs.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
            model_crs.add(MaxPooling1D(pool_size=3))
            BatchNormalization()
            model_crs.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
            model_crs.add(MaxPooling1D(pool_size=3))
            '''model_crs.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
            model_crs.add(MaxPooling1D(pool_size=3))
            model_crs.add(Conv1D(filters=1024, kernel_size=3, activation='relu'))
            model_crs.add(MaxPooling1D(pool_size=3))
            '''
            model_crs.add(Flatten())
            #model_crs.add(Dropout(0.5))
            model_crs.add(Dense(2000, activation='relu'))
            model_crs.add(Dense(n_outputs, activation='softmax'))
            model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
            # fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy
            earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 20, restore_best_weights = True)
            model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])
            # evaluate model_crs
            _, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=verbose)
            predict_x=model_crs.predict(X_test)              # Function 1
            y_pred_crs=maxindx(predict_x)                    # function from augment.py to remove 0 index predictions
            rprt0=classification_report(y_test, y_pred_crs,output_dict=True)
            rprt_row=np.array([])
            rprt_row=np.append(rprt_row,rprt0['accuracy'])
            rprt_row=np.append(rprt_row,rprt0['macro avg']['f1-score'])
            rprt_row=np.append(rprt_row,rprt0['weighted avg']['f1-score'])
            rprt=np.append(rprt,[rprt_row],axis=0)
        #print('rprt=',rprt)
        maxm=rprt[np.argmax(rprt[:,1])]
        #print('maxm=',maxm)
        avrg=np.mean(rprt,axis=0)
        #print('avrg=',avrg)
        rslts=np.append(rslts,['Beta '+str(beta)+'  alpha '+str(alpha) +'  average= '+str(np.int16(100*avrg))+'  max= '+str(np.int16(100*maxm))+'  var= '+str(np.int16(np.var(100*rprt,axis=0)))],axis=0)
        #print('rslts=',rslts)


 max = 32
[[4 3 0 2 2 2 2]
 [3 2 0 1 1 1 1]
 [5 4 0 2 2 2 2]
 [3 3 0 1 1 1 1]
 [1 1 0 0 0 0 0]
 [2 1 0 1 1 1 1]
 [2 1 0 1 1 1 1]
 [2 2 0 1 1 1 1]
 [3 2 0 1 1 1 1]
 [2 1 0 1 1 1 1]
 [4 3 0 2 2 2 2]
 [2 1 0 1 1 1 1]]
12/12 [==============================] - 0s 4ms/step
[[11  9  0  4  4  4  4]
 [ 7  5  0  3  3  3  3]
 [13 11  0  5  5  5  5]
 [ 9  7  0  4  4  4  4]
 [ 2  2  0  1  1  1  1]
 [ 5  4  0  2  2  2  2]
 [ 5  4  0  2  2  2  2]
 [ 5  4  0  2  2  2  2]
 [ 7  5  0  3  3  3  3]
 [ 4  3  0  2  2  2  2]
 [11  9  0  4  4  4  4]
 [ 3  3  0  1  1  1  1]]
12/12 [==============================] - 0s 3ms/step
[[34 28  0 14 14 14 14]
 [21 17  0  9  9  9  9]
 [43 34  0 17 17 17 17]
 [29 23  0 12 12 12 12]
 [ 6  5  0  2  2  2  2]
 [14 12  0  6  6  6  6]
 [15 12  0  6  6  6  6]
 [15 12  0  6  6  6  6]
 [21 17  0  9  9  9  9]
 [12 10  0  5  5  5  5]
 [35 28  0 14 14 14 14]
 [10  8  0  4  4  4  4]]
12/12 [==============================] - 0s 3ms/step
[[6 4 0 1 1 1 1]
 [4 3 0 1 1 1 1]
 [7 5 0 1 1 1

In [20]:
aug_amnt_Ttl

array([[  4,   3,   0,   2,   2,   2,   2],
       [  3,   2,   0,   1,   1,   1,   1],
       [  5,   4,   0,   2,   2,   2,   2],
       [  3,   3,   0,   1,   1,   1,   1],
       [  1,   1,   0,   0,   0,   0,   0],
       [  2,   1,   0,   1,   1,   1,   1],
       [  2,   1,   0,   1,   1,   1,   1],
       [  2,   2,   0,   1,   1,   1,   1],
       [  3,   2,   0,   1,   1,   1,   1],
       [  2,   1,   0,   1,   1,   1,   1],
       [  4,   3,   0,   2,   2,   2,   2],
       [  2,   1,   0,   1,   1,   1,   1],
       [ 11,   9,   0,   4,   4,   4,   4],
       [  7,   5,   0,   3,   3,   3,   3],
       [ 13,  11,   0,   5,   5,   5,   5],
       [  9,   7,   0,   4,   4,   4,   4],
       [  2,   2,   0,   1,   1,   1,   1],
       [  5,   4,   0,   2,   2,   2,   2],
       [  5,   4,   0,   2,   2,   2,   2],
       [  5,   4,   0,   2,   2,   2,   2],
       [  7,   5,   0,   3,   3,   3,   3],
       [  4,   3,   0,   2,   2,   2,   2],
       [ 11,   9,   0,   4,   4,

In [21]:
rslts

array(['Beta 1  alpha 1  average= [86 86 86]  max= [87 87 87]  var= [2 2 2]',
       'Beta 1  alpha 3  average= [86 86 86]  max= [89 89 89]  var= [3 3 3]',
       'Beta 1  alpha 10  average= [87 87 87]  max= [89 89 89]  var= [2 2 2]',
       'Beta 2  alpha 1  average= [84 84 84]  max= [86 86 86]  var= [5 4 4]',
       'Beta 2  alpha 3  average= [86 86 86]  max= [89 89 89]  var= [1 1 1]',
       'Beta 2  alpha 10  average= [87 87 87]  max= [89 89 89]  var= [1 1 1]',
       'Beta 6  alpha 1  average= [83 83 83]  max= [86 86 86]  var= [3 3 3]',
       'Beta 6  alpha 3  average= [86 85 85]  max= [87 87 87]  var= [1 1 1]',
       'Beta 6  alpha 10  average= [86 86 86]  max= [88 88 88]  var= [3 3 3]'],
      dtype='<U68')

In [22]:
results3=rslts

In [23]:
mx_wndws=int(np.max(cls_wndws)+1)   # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max =', mx_wndws)
aug_amnt_Ttl=np.empty((0,7), int) 
rslts=np.array([])
X=1                               #داده افزایی تا چند برابر کلاس اکثریت
for beta in range(1,4):
    if beta==3:
        beta=beta*2
    rprt=np.array([])
    for alpha in range(1,4):    #چند برابر کردن داده افزایی هر کلاس متناسب ضعیف بودن آن کلاس
        scors0=np.array([64,78,55,70,94,85,84,85,78,88,63,90])         #Class Recalls without Augmentation
        AgScr=1+alpha*((100-scors0)/(np.max(100-scors0)))**beta            #Class Specific

        f_scr=np.array([83,88,87,83,85,85,85,85])                      #UnAg,Scl.2,Mag.05,Tm.2,Gs1,frqnc2,GAN,LSTMexpgdo
        f_scr_difrnc=f_scr-f_scr[0]
        for z in range (len(f_scr_difrnc)):
            if f_scr_difrnc[z]<0:
                f_scr_difrnc[z]=0
        f_scr_difrnc=f_scr_difrnc**beta
        f_scr_rtio=f_scr_difrnc/(sum(f_scr_difrnc))
        aug_amnt=np.empty((0,len(f_scr_rtio)-1), int) 

        for i in range (1,cls_num+1):
            aug_amnt=np.append(aug_amnt,[f_scr_rtio[1:]*(mx_wndws*AgScr[i-1]-cls_wndws[i-1])],axis=0)
            #vlum_win=mx_wndws-cls_wndws[i-1]             #میزان داده افزایی در روش های عمیق (پنجره)
        aug_amnt=np.int16(np.round(aug_amnt))
        aug_amnt_Ttl=np.append(aug_amnt_Ttl,aug_amnt,axis=0)
        #print(np.int16(100*f_scr_rtio))
        #print(aug_amnt)
        rprt=np.empty((0,3),float)
        print(aug_amnt)
        for repeat in range(1,rpt_cnn_clsfctn+1):
            xtrain=np.empty((0,smpl_rte+1), float) 
            for i in range (1,cls_num+1):                                     #تعریف آرایه ی پنجره های کلاس ها
                xtrain=np.append(xtrain,vars()['wndws'+str(i)],axis=0)
                if aug_amnt[i-1,0]>0:
                    methd=10
                    sort=-3  #-5=Dis_sam_Cls=qlty1 #-4=var_Dis=qlty2 #-3=DisSm/DisOthr=qlty3  #-2=invrs_GDO_papr=qlty4   #-1=GDO_papr=qlty5
                    dp_slct_dstrb=4            #1-sequencial 2-linear 3-beta 4-exponential 5-unfrm
                    Hrzntl=srtd_dataH(methd,i,aug_amnt[i-1,0],dp_slct_dstrb,sort)
                    Vrtcl=srtd_dataV(methd,i,aug_amnt[i-1,0],dp_slct_dstrb,sort)
                    HrzntlVrtcl=np.concatenate((Hrzntl[:,:-1],Vrtcl),axis=1)
                    xtrain=np.append(xtrain,HrzntlVrtcl,axis=0)
                if aug_amnt[i-1,1]>0:
                    methd=20
                    sort=-3
                    dp_slct_dstrb=2
                    Hrzntl=srtd_dataH(methd,i,aug_amnt[i-1,1],dp_slct_dstrb,sort)
                    Vrtcl=srtd_dataV(methd,i,aug_amnt[i-1,1],dp_slct_dstrb,sort)
                    HrzntlVrtcl=np.concatenate((Hrzntl[:,:-1],Vrtcl),axis=1)
                    xtrain=np.append(xtrain,HrzntlVrtcl,axis=0)
                if aug_amnt[i-1,2]>0:
                    methd=30
                    sort=-3
                    dp_slct_dstrb=4
                    Hrzntl=srtd_dataH(methd,i,aug_amnt[i-1,2],dp_slct_dstrb,sort)
                    Vrtcl=srtd_dataV(methd,i,aug_amnt[i-1,2],dp_slct_dstrb,sort)
                    HrzntlVrtcl=np.concatenate((Hrzntl[:,:-1],Vrtcl),axis=1)
                    xtrain=np.append(xtrain,HrzntlVrtcl,axis=0)
                if aug_amnt[i-1,3]>0:
                    methd=40
                    sort=-1
                    dp_slct_dstrb=4
                    Hrzntl=srtd_dataH(methd,i,aug_amnt[i-1,3],dp_slct_dstrb,sort)
                    Vrtcl=srtd_dataV(methd,i,aug_amnt[i-1,3],dp_slct_dstrb,sort)
                    HrzntlVrtcl=np.concatenate((Hrzntl[:,:-1],Vrtcl),axis=1)
                    xtrain=np.append(xtrain,HrzntlVrtcl,axis=0)
                if aug_amnt[i-1,4]>0:
                    methd=50
                    sort=-3
                    dp_slct_dstrb=4
                    Hrzntl=srtd_dataH(methd,i,aug_amnt[i-1,4],dp_slct_dstrb,sort)
                    Vrtcl=srtd_dataV(methd,i,aug_amnt[i-1,4],dp_slct_dstrb,sort)
                    HrzntlVrtcl=np.concatenate((Hrzntl[:,:-1],Vrtcl),axis=1)
                    xtrain=np.append(xtrain,HrzntlVrtcl,axis=0)
                if aug_amnt[i-1,5]>0:
                    methd=60
                    sort=-1
                    dp_slct_dstrb=4
                    Hrzntl=srtd_dataH(methd,i,aug_amnt[i-1,5],dp_slct_dstrb,sort)
                    Vrtcl=srtd_dataV(methd,i,aug_amnt[i-1,5],dp_slct_dstrb,sort)
                    HrzntlVrtcl=np.concatenate((Hrzntl[:,:-1],Vrtcl),axis=1)
                    xtrain=np.append(xtrain,HrzntlVrtcl,axis=0)
                if aug_amnt[i-1,6]>0:
                    methd=70
                    sort=-1
                    dp_slct_dstrb=2
                    Hrzntl=srtd_dataH(methd,i,aug_amnt[i-1,6],dp_slct_dstrb,sort)
                    Vrtcl=srtd_dataV(methd,i,aug_amnt[i-1,6],dp_slct_dstrb,sort)
                    HrzntlVrtcl=np.concatenate((Hrzntl[:,:-1],Vrtcl),axis=1)
                    xtrain=np.append(xtrain,HrzntlVrtcl,axis=0)
                #print('class ', i, 'train shape = ', np.shape(xtrain))

            # Normalization train windows
            mx_aug=np.max(xtrain[:,:-1])
            mx=np.max(ecg[:,:-1])
            mn_aug=np.min(xtrain[:,:-1])
            mn=np.min(ecg[:,:-1])
            xtrain[:,:-1]= 2*(xtrain[:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1


            ###Test Windows:
            wndws_test=np.empty((0,len(wndws1[0])), float)
            i=0
            windws=np.array([])
            for cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های آموزش اصلی
                vars()['wndws_tst'+str(cls)]=np.empty((0,len(wndws1[0])), float)
                vars()['wndws_tst'+str(cls)]=np.append(vars()['wndws_tst'+str(cls)],vars()['ecg_tst'+str(cls)],axis=0)   

            # Normalization test windows
            for i in range (1,cls_num+1):                
                cls=i
                #print('\n cls', i, ' >> ')
                #print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
                #print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
                vars()['wndws_tst'+str(i)][:,:-1]= 2*(vars()['wndws_tst'+str(i)][:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1
                #print('after normalizing >>')
                #print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
                #print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
            #print("each class and its windows =")                    #تجمیع کلاس های تست
            for cls in range (1,cls_num+1):                                # آرایه ی تعداد پنجره ی هر کلاس
                wns=len(vars()['wndws_tst'+str(cls)])
                wndws_test=np.append(wndws_test,vars()['wndws_tst'+str(cls)],axis=0)
            Xtest=wndws_test[:,:-1]
            ytest=np.int16(wndws_test[:,-1])

            trainx=np.random.permutation(xtrain)
            Xtrain=np.array(trainx[:,:-1])
            ytrain=np.int16(trainx[:,-1])

            X_train=np.array(Xtrain)
            y_train=np.array(ytrain)
            X_test=np.array(Xtest)
            y_test=np.array(ytest)
            X_valid=np.array(Xtrain)
            y_valid=np.array(ytrain)
            #print('X_train => ', X_train.shape)
            #print('y_train => ', y_train.shape)
            #print('X_test  => ', X_test.shape)
            #print('y_test  => ', y_test.shape)
            #print('X_valid  => ', X_valid.shape)
            #print('y_valid  => ', y_valid.shape)

            y_train_cat=to_categorical(y_train)
            y_valid_cat=to_categorical(y_valid)
            y_test_cat=to_categorical(y_test)
            X_train = np.expand_dims(X_train, axis=2)
            X_valid = np.expand_dims(X_valid, axis=2)
            X_test = np.expand_dims(X_test, axis=2)

            ####### NETWORK #######
            accuracy=0
            acc_crs=np.array([])
            ##for i in range (10):
            ##    if accuracy<0.2 :
            verbose, epochs, batch_size = 0, 1000, btch
            n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
            steps_per_epoch = len(X_train)//batch_size
            validation_steps = len(X_valid)//batch_size # if you have test data
            model_crs = Sequential()
            BatchNormalization()
            model_crs.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
            #model_crs.add(MaxPooling1D(pool_size=3))
            BatchNormalization()
            model_crs.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
            #model_crs.add(Dropout(0.5))
            model_crs.add(MaxPooling1D(pool_size=3))
            BatchNormalization()
            model_crs.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
            model_crs.add(MaxPooling1D(pool_size=3))
            BatchNormalization()
            model_crs.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
            model_crs.add(MaxPooling1D(pool_size=3))
            BatchNormalization()
            model_crs.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
            model_crs.add(MaxPooling1D(pool_size=3))
            '''model_crs.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
            model_crs.add(MaxPooling1D(pool_size=3))
            model_crs.add(Conv1D(filters=1024, kernel_size=3, activation='relu'))
            model_crs.add(MaxPooling1D(pool_size=3))
            '''
            model_crs.add(Flatten())
            #model_crs.add(Dropout(0.5))
            model_crs.add(Dense(2000, activation='relu'))
            model_crs.add(Dense(n_outputs, activation='softmax'))
            model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
            # fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy
            earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 20, restore_best_weights = True)
            model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])
            # evaluate model_crs
            _, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=verbose)
            predict_x=model_crs.predict(X_test)              # Function 1
            y_pred_crs=maxindx(predict_x)                    # function from augment.py to remove 0 index predictions
            rprt0=classification_report(y_test, y_pred_crs,output_dict=True)
            rprt_row=np.array([])
            rprt_row=np.append(rprt_row,rprt0['accuracy'])
            rprt_row=np.append(rprt_row,rprt0['macro avg']['f1-score'])
            rprt_row=np.append(rprt_row,rprt0['weighted avg']['f1-score'])
            rprt=np.append(rprt,[rprt_row],axis=0)
        #print('rprt=',rprt)
        maxm=rprt[np.argmax(rprt[:,1])]
        #print('maxm=',maxm)
        avrg=np.mean(rprt,axis=0)
        #print('avrg=',avrg)
        rslts=np.append(rslts,['X '+str(X)+'  Beta '+str(beta)+'  alpha '+str(alpha) +'  average= '+str(np.int16(100*avrg))+'  max= '+str(np.int16(100*maxm))+'  var= '+str(np.int16(np.var(100*rprt,axis=0)))],axis=0)
        #print('rslts=',rslts)


 max = 32
[[ 8  6  0  3  3  3  3]
 [ 5  4  0  2  2  2  2]
 [10  8  0  4  4  4  4]
 [ 7  5  0  3  3  3  3]
 [ 2  1  0  1  1  1  1]
 [ 3  3  0  1  1  1  1]
 [ 4  3  0  1  1  1  1]
 [ 4  3  0  1  1  1  1]
 [ 5  4  0  2  2  2  2]
 [ 3  2  0  1  1  1  1]
 [ 8  7  0  3  3  3  3]
 [ 3  2  0  1  1  1  1]]
12/12 [==============================] - 0s 3ms/step
[[16 13  0  6  6  6  6]
 [10  8  0  4  4  4  4]
 [19 16  0  8  8  8  8]
 [13 11  0  5  5  5  5]
 [ 3  2  0  1  1  1  1]
 [ 7  5  0  3  3  3  3]
 [ 7  6  0  3  3  3  3]
 [ 7  5  0  3  3  3  3]
 [10  8  0  4  4  4  4]
 [ 6  4  0  2  2  2  2]
 [16 13  0  6  6  6  6]
 [ 5  4  0  2  2  2  2]]
12/12 [==============================] - 0s 3ms/step
[[23 19  0  9  9  9  9]
 [14 12  0  6  6  6  6]
 [29 23  0 12 12 12 12]
 [19 16  0  8  8  8  8]
 [ 4  3  0  2  2  2  2]
 [10  8  0  4  4  4  4]
 [10  8  0  4  4  4  4]
 [10  8  0  4  4  4  4]
 [14 12  0  6  6  6  6]
 [ 8  6  0  3  3  3  3]
 [24 19  0 10 10 10 10]
 [ 7  5  0  3  3  3  3]]
12/12 [=========

In [24]:
aug_amnt_Ttl

array([[ 8,  6,  0,  3,  3,  3,  3],
       [ 5,  4,  0,  2,  2,  2,  2],
       [10,  8,  0,  4,  4,  4,  4],
       [ 7,  5,  0,  3,  3,  3,  3],
       [ 2,  1,  0,  1,  1,  1,  1],
       [ 3,  3,  0,  1,  1,  1,  1],
       [ 4,  3,  0,  1,  1,  1,  1],
       [ 4,  3,  0,  1,  1,  1,  1],
       [ 5,  4,  0,  2,  2,  2,  2],
       [ 3,  2,  0,  1,  1,  1,  1],
       [ 8,  7,  0,  3,  3,  3,  3],
       [ 3,  2,  0,  1,  1,  1,  1],
       [16, 13,  0,  6,  6,  6,  6],
       [10,  8,  0,  4,  4,  4,  4],
       [19, 16,  0,  8,  8,  8,  8],
       [13, 11,  0,  5,  5,  5,  5],
       [ 3,  2,  0,  1,  1,  1,  1],
       [ 7,  5,  0,  3,  3,  3,  3],
       [ 7,  6,  0,  3,  3,  3,  3],
       [ 7,  5,  0,  3,  3,  3,  3],
       [10,  8,  0,  4,  4,  4,  4],
       [ 6,  4,  0,  2,  2,  2,  2],
       [16, 13,  0,  6,  6,  6,  6],
       [ 5,  4,  0,  2,  2,  2,  2],
       [23, 19,  0,  9,  9,  9,  9],
       [14, 12,  0,  6,  6,  6,  6],
       [29, 23,  0, 12, 12, 12, 12],
 

In [25]:
rslts

array(['Beta 1  alpha 1  average= [86 86 86]  max= [88 88 88]  var= [1 2 2]',
       'Beta 1  alpha 2  average= [85 85 85]  max= [85 86 85]  var= [0 0 0]',
       'Beta 1  alpha 3  average= [87 87 87]  max= [88 88 88]  var= [2 2 2]',
       'Beta 2  alpha 1  average= [85 85 85]  max= [89 89 89]  var= [6 6 6]',
       'Beta 2  alpha 2  average= [85 85 85]  max= [87 87 87]  var= [1 1 1]',
       'Beta 2  alpha 3  average= [86 86 86]  max= [87 87 87]  var= [0 0 0]',
       'Beta 6  alpha 1  average= [84 84 84]  max= [86 85 85]  var= [0 1 1]',
       'Beta 6  alpha 2  average= [85 85 85]  max= [86 85 85]  var= [0 0 0]',
       'Beta 6  alpha 3  average= [84 84 84]  max= [86 86 86]  var= [1 1 1]'],
      dtype='<U67')

In [26]:
results4=rslts

In [30]:
mx_wndws=int(np.max(cls_wndws)+1)   # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max =', mx_wndws)
rpt_cnn_clsfctn=3                 #تعداد اجرای شبکه عصبی برای میانگین گیری هر حالت 
aug_amnt_Ttl=np.empty((0,7), int) 
rslts=np.array([])
for X in range(1,4):                               #داده افزایی تا چند برابر کلاس اکثریت
    mx_wndws=int(np.max(cls_wndws))   # حداکثر تعداد پنجره ی موجود بین کلاس ها
    mx_wndws=int(X*mx_wndws)          #
    for alpha in range(1,4):
        rprt=np.array([])
        for beta in range(1,4):                        #چند برابر کردن داده افزایی هر کلاس متناسب ضعیف بودن آن کلاس
            if beta==3:
                beta=beta*2
            scors0=np.array([64,78,55,70,94,85,84,85,78,88,63,90])         #Class Recalls without Augmentation
            AgScr=1+alpha*((100-scors0)/(np.max(100-scors0)))**beta        #Class Specific

            f_scr=np.array([83,88,87,83,85,85,85,85])                      #UnAg,Scl.2,Mag.05,Tm.2,Gs1,frqnc2,GAN,LSTMexpgdo
            f_scr_difrnc=f_scr-f_scr[0]
            for z in range (len(f_scr_difrnc)):
                if f_scr_difrnc[z]<0:
                    f_scr_difrnc[z]=0
            f_scr_difrnc=f_scr_difrnc**beta
            f_scr_rtio=f_scr_difrnc/(sum(f_scr_difrnc))
            aug_amnt=np.empty((0,len(f_scr_rtio)-1), int) 

            for i in range (1,cls_num+1):
                aug_amnt=np.append(aug_amnt,[f_scr_rtio[1:]*(mx_wndws*AgScr[i-1]-cls_wndws[i-1])],axis=0)
                #vlum_win=mx_wndws-cls_wndws[i-1]                        #میزان داده افزایی در روش های عمیق (پنجره)
            aug_amnt=np.int16(np.round(aug_amnt))
            aug_amnt_Ttl=np.append(aug_amnt_Ttl,aug_amnt,axis=0)
            #print(np.int16(100*f_scr_rtio))
            #print(aug_amnt)
            rprt=np.empty((0,3),float)
            print(aug_amnt)
            for repeat in range(1,rpt_cnn_clsfctn+1):
                xtrain=np.empty((0,smpl_rte+1), float) 
                for i in range (1,cls_num+1):                            #تعریف آرایه ی پنجره های کلاس ها
                    xtrain=np.append(xtrain,vars()['wndws'+str(i)],axis=0)
                    if aug_amnt[i-1,0]>0:
                        methd=10
                        sort=-3  #-5=Dis_sam_Cls=qlty1 #-4=var_Dis=qlty2 #-3=DisSm/DisOthr=qlty3  #-2=invrs_GDO_papr=qlty4   #-1=GDO_papr=qlty5
                        dp_slct_dstrb=4            #1-sequencial 2-linear 3-beta 4-exponential 5-unfrm
                        Hrzntl=srtd_dataH(methd,i,aug_amnt[i-1,0],dp_slct_dstrb,sort)
                        Vrtcl=srtd_dataV(methd,i,aug_amnt[i-1,0],dp_slct_dstrb,sort)
                        HrzntlVrtcl=np.concatenate((Hrzntl[:,:-1],Vrtcl),axis=1)
                        xtrain=np.append(xtrain,HrzntlVrtcl,axis=0)
                    if aug_amnt[i-1,1]>0:
                        methd=20
                        sort=-3
                        dp_slct_dstrb=2
                        Hrzntl=srtd_dataH(methd,i,aug_amnt[i-1,1],dp_slct_dstrb,sort)
                        Vrtcl=srtd_dataV(methd,i,aug_amnt[i-1,1],dp_slct_dstrb,sort)
                        HrzntlVrtcl=np.concatenate((Hrzntl[:,:-1],Vrtcl),axis=1)
                        xtrain=np.append(xtrain,HrzntlVrtcl,axis=0)
                    if aug_amnt[i-1,2]>0:
                        methd=30
                        sort=-3
                        dp_slct_dstrb=4
                        Hrzntl=srtd_dataH(methd,i,aug_amnt[i-1,2],dp_slct_dstrb,sort)
                        Vrtcl=srtd_dataV(methd,i,aug_amnt[i-1,2],dp_slct_dstrb,sort)
                        HrzntlVrtcl=np.concatenate((Hrzntl[:,:-1],Vrtcl),axis=1)
                        xtrain=np.append(xtrain,HrzntlVrtcl,axis=0)
                    if aug_amnt[i-1,3]>0:
                        methd=40
                        sort=-1
                        dp_slct_dstrb=4
                        Hrzntl=srtd_dataH(methd,i,aug_amnt[i-1,3],dp_slct_dstrb,sort)
                        Vrtcl=srtd_dataV(methd,i,aug_amnt[i-1,3],dp_slct_dstrb,sort)
                        HrzntlVrtcl=np.concatenate((Hrzntl[:,:-1],Vrtcl),axis=1)
                        xtrain=np.append(xtrain,HrzntlVrtcl,axis=0)
                    if aug_amnt[i-1,4]>0:
                        methd=50
                        sort=-3
                        dp_slct_dstrb=4
                        Hrzntl=srtd_dataH(methd,i,aug_amnt[i-1,4],dp_slct_dstrb,sort)
                        Vrtcl=srtd_dataV(methd,i,aug_amnt[i-1,4],dp_slct_dstrb,sort)
                        HrzntlVrtcl=np.concatenate((Hrzntl[:,:-1],Vrtcl),axis=1)
                        xtrain=np.append(xtrain,HrzntlVrtcl,axis=0)
                    if aug_amnt[i-1,5]>0:
                        methd=60
                        sort=-1
                        dp_slct_dstrb=4
                        Hrzntl=srtd_dataH(methd,i,aug_amnt[i-1,5],dp_slct_dstrb,sort)
                        Vrtcl=srtd_dataV(methd,i,aug_amnt[i-1,5],dp_slct_dstrb,sort)
                        HrzntlVrtcl=np.concatenate((Hrzntl[:,:-1],Vrtcl),axis=1)
                        xtrain=np.append(xtrain,HrzntlVrtcl,axis=0)
                    if aug_amnt[i-1,6]>0:
                        methd=70
                        sort=-1
                        dp_slct_dstrb=2
                        Hrzntl=srtd_dataH(methd,i,aug_amnt[i-1,6],dp_slct_dstrb,sort)
                        Vrtcl=srtd_dataV(methd,i,aug_amnt[i-1,6],dp_slct_dstrb,sort)
                        HrzntlVrtcl=np.concatenate((Hrzntl[:,:-1],Vrtcl),axis=1)
                        xtrain=np.append(xtrain,HrzntlVrtcl,axis=0)
                    #print('class ', i, 'train shape = ', np.shape(xtrain))

                # Normalization train windows
                mx_aug=np.max(xtrain[:,:-1])
                mx=np.max(ecg[:,:-1])
                mn_aug=np.min(xtrain[:,:-1])
                mn=np.min(ecg[:,:-1])
                xtrain[:,:-1]= 2*(xtrain[:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1


                ###Test Windows:
                wndws_test=np.empty((0,len(wndws1[0])), float)
                i=0
                windws=np.array([])
                for cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های آموزش اصلی
                    vars()['wndws_tst'+str(cls)]=np.empty((0,len(wndws1[0])), float)
                    vars()['wndws_tst'+str(cls)]=np.append(vars()['wndws_tst'+str(cls)],vars()['ecg_tst'+str(cls)],axis=0)   

                # Normalization test windows
                for i in range (1,cls_num+1):                
                    cls=i
                    #print('\n cls', i, ' >> ')
                    #print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
                    #print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
                    vars()['wndws_tst'+str(i)][:,:-1]= 2*(vars()['wndws_tst'+str(i)][:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1
                    #print('after normalizing >>')
                    #print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
                    #print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
                #print("each class and its windows =")                    #تجمیع کلاس های تست
                for cls in range (1,cls_num+1):                                # آرایه ی تعداد پنجره ی هر کلاس
                    wns=len(vars()['wndws_tst'+str(cls)])
                    wndws_test=np.append(wndws_test,vars()['wndws_tst'+str(cls)],axis=0)
                Xtest=wndws_test[:,:-1]
                ytest=np.int16(wndws_test[:,-1])

                trainx=np.random.permutation(xtrain)
                Xtrain=np.array(trainx[:,:-1])
                ytrain=np.int16(trainx[:,-1])

                X_train=np.array(Xtrain)
                y_train=np.array(ytrain)
                X_test=np.array(Xtest)
                y_test=np.array(ytest)
                X_valid=np.array(Xtrain)
                y_valid=np.array(ytrain)
                #print('X_train => ', X_train.shape)
                #print('y_train => ', y_train.shape)
                #print('X_test  => ', X_test.shape)
                #print('y_test  => ', y_test.shape)
                #print('X_valid  => ', X_valid.shape)
                #print('y_valid  => ', y_valid.shape)

                y_train_cat=to_categorical(y_train)
                y_valid_cat=to_categorical(y_valid)
                y_test_cat=to_categorical(y_test)
                X_train = np.expand_dims(X_train, axis=2)
                X_valid = np.expand_dims(X_valid, axis=2)
                X_test = np.expand_dims(X_test, axis=2)

                ####### NETWORK #######
                accuracy=0
                acc_crs=np.array([])
                ##for i in range (10):
                ##    if accuracy<0.2 :
                verbose, epochs, batch_size = 0, 1000, btch
                n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
                steps_per_epoch = len(X_train)//batch_size
                validation_steps = len(X_valid)//batch_size # if you have test data
                model_crs = Sequential()
                BatchNormalization()
                model_crs.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
                #model_crs.add(MaxPooling1D(pool_size=3))
                BatchNormalization()
                model_crs.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
                #model_crs.add(Dropout(0.5))
                model_crs.add(MaxPooling1D(pool_size=3))
                BatchNormalization()
                model_crs.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
                model_crs.add(MaxPooling1D(pool_size=3))
                BatchNormalization()
                model_crs.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
                model_crs.add(MaxPooling1D(pool_size=3))
                BatchNormalization()
                model_crs.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
                model_crs.add(MaxPooling1D(pool_size=3))
                '''model_crs.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
                model_crs.add(MaxPooling1D(pool_size=3))
                model_crs.add(Conv1D(filters=1024, kernel_size=3, activation='relu'))
                model_crs.add(MaxPooling1D(pool_size=3))
                '''
                model_crs.add(Flatten())
                #model_crs.add(Dropout(0.5))
                model_crs.add(Dense(2000, activation='relu'))
                model_crs.add(Dense(n_outputs, activation='softmax'))
                model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
                # fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy
                earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 20, restore_best_weights = True)
                model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])
                # evaluate model_crs
                _, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=verbose)
                predict_x=model_crs.predict(X_test)              # Function 1
                y_pred_crs=maxindx(predict_x)                    # function from augment.py to remove 0 index predictions
                rprt0=classification_report(y_test, y_pred_crs,output_dict=True)
                rprt_row=np.array([])
                rprt_row=np.append(rprt_row,rprt0['accuracy'])
                rprt_row=np.append(rprt_row,rprt0['macro avg']['f1-score'])
                rprt_row=np.append(rprt_row,rprt0['weighted avg']['f1-score'])
                rprt=np.append(rprt,[rprt_row],axis=0)
            #print('rprt=',rprt)
            maxm=rprt[np.argmax(rprt[:,1])]
            #print('maxm=',maxm)
            avrg=np.mean(rprt,axis=0)
            #print('avrg=',avrg)
            rslts=np.append(rslts,['X '+str(X)+'  alpha '+str(alpha) +'  Beta '+str(beta)+'  average= '+str(np.int16(100*avrg))+'  max= '+str(np.int16(100*maxm))+'  var= '+str(np.int16(np.var(100*rprt,axis=0)))],axis=0)
            print('rslts=',rslts[-1])


 max = 32
[[8 6 0 3 3 3 3]
 [5 4 0 2 2 2 2]
 [9 8 0 4 4 4 4]
 [6 5 0 3 3 3 3]
 [2 1 0 1 1 1 1]
 [3 2 0 1 1 1 1]
 [3 3 0 1 1 1 1]
 [3 3 0 1 1 1 1]
 [5 4 0 2 2 2 2]
 [3 2 0 1 1 1 1]
 [8 6 0 3 3 3 3]
 [2 2 0 1 1 1 1]]
12/12 [==============================] - 0s 4ms/step
rslts= X 1  alpha 1  Beta 1  average= [85 85 85]  max= [86 86 86]  var= [1 0 0]
[[ 9  6  0  1  1  1  1]
 [ 4  2  0  1  1  1  1]
 [14  9  0  2  2  2  2]
 [ 6  4  0  1  1  1  1]
 [ 1  0  0  0  0  0  0]
 [ 2  1  0  0  0  0  0]
 [ 2  1  0  0  0  0  0]
 [ 2  1  0  0  0  0  0]
 [ 4  2  0  1  1  1  1]
 [ 1  1  0  0  0  0  0]
 [10  6  0  2  2  2  2]
 [ 1  1  0  0  0  0  0]]
12/12 [==============================] - 0s 3ms/step
rslts= X 1  alpha 1  Beta 2  average= [59 56 56]  max= [84 84 84]  var= [1277 1514 1512]
[[ 7  2  0  0  0  0  0]
 [ 1  0  0  0  0  0  0]
 [25  7  0  0  0  0  0]
 [ 3  1  0  0  0  0  0]
 [ 1  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 1  0  0  0  0  0  0]
 [ 1  0  0  0  0  0  0]
 [ 1

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


12/12 [==============================] - 0s 3ms/step
rslts= X 1  alpha 1  Beta 6  average= [82 82 82]  max= [85 84 84]  var= [2 3 3]
[[15 12  0  6  6  6  6]
 [ 9  7  0  4  4  4  4]
 [19 15  0  7  7  7  7]
 [12 10  0  5  5  5  5]
 [ 3  2  0  1  1  1  1]
 [ 6  5  0  2  2  2  2]
 [ 6  5  0  3  3  3  3]
 [ 6  5  0  3  3  3  3]
 [ 9  7  0  4  4  4  4]
 [ 5  4  0  2  2  2  2]
 [15 12  0  6  6  6  6]
 [ 4  3  0  2  2  2  2]]
12/12 [==============================] - 0s 3ms/step
rslts= X 1  alpha 2  Beta 1  average= [85 85 85]  max= [85 85 85]  var= [0 0 0]
[[18 11  0  3  3  3  3]
 [ 7  4  0  1  1  1  1]
 [28 18  0  4  4  4  4]
 [13  8  0  2  2  2  2]
 [ 1  1  0  0  0  0  0]
 [ 3  2  0  0  0  0  0]
 [ 3  2  0  1  1  1  1]
 [ 3  2  0  1  1  1  1]
 [ 7  4  0  1  1  1  1]
 [ 2  2  0  0  0  0  0]
 [19 12  0  3  3  3  3]
 [ 2  1  0  0  0  0  0]]
12/12 [==============================] - 0s 3ms/step
rslts= X 1  alpha 2  Beta 2  average= [87 87 87]  max= [87 87 87]  var= [0 0 0]
[[13  4  0  0  0  0  0]

12/12 [==============================] - 0s 3ms/step
rslts= X 2  alpha 3  Beta 6  average= [87 87 87]  max= [89 89 89]  var= [1 1 1]
[[40 32  0 16 16 16 16]
 [32 26  0 13 13 13 13]
 [46 37  0 18 18 18 18]
 [37 29  0 15 15 15 15]
 [22 18  0  9  9  9  9]
 [27 22  0 11 11 11 11]
 [28 22  0 11 11 11 11]
 [28 22  0 11 11 11 11]
 [32 26  0 13 13 13 13]
 [26 21  0 10 10 10 10]
 [41 33  0 16 16 16 16]
 [25 20  0 10 10 10 10]]
12/12 [==============================] - 0s 3ms/step
rslts= X 3  alpha 1  Beta 1  average= [87 87 87]  max= [90 90 90]  var= [3 3 3]
[[54 34  0  9  9  9  9]
 [37 24  0  6  6  6  6]
 [68 44  0 11 11 11 11]
 [46 29  0  7  7  7  7]
 [28 18  0  5  5  5  5]
 [32 20  0  5  5  5  5]
 [32 21  0  5  5  5  5]
 [32 21  0  5  5  5  5]
 [37 24  0  6  6  6  6]
 [31 20  0  5  5  5  5]
 [55 35  0  9  9  9  9]
 [30 19  0  5  5  5  5]]
12/12 [==============================] - 0s 3ms/step
rslts= X 3  alpha 1  Beta 2  average= [88 88 88]  max= [90 90 90]  var= [1 1 1]
[[ 68  18   0   0   0  

In [31]:
rslts

array(['X 1  alpha 1  Beta 1  average= [85 85 85]  max= [86 86 86]  var= [1 0 0]',
       'X 1  alpha 1  Beta 2  average= [59 56 56]  max= [84 84 84]  var= [1277 1514 1512]',
       'X 1  alpha 1  Beta 6  average= [82 82 82]  max= [85 84 84]  var= [2 3 3]',
       'X 1  alpha 2  Beta 1  average= [85 85 85]  max= [85 85 85]  var= [0 0 0]',
       'X 1  alpha 2  Beta 2  average= [87 87 87]  max= [87 87 87]  var= [0 0 0]',
       'X 1  alpha 2  Beta 6  average= [84 84 84]  max= [85 85 85]  var= [0 0 0]',
       'X 1  alpha 3  Beta 1  average= [87 87 86]  max= [88 88 88]  var= [1 1 1]',
       'X 1  alpha 3  Beta 2  average= [84 84 84]  max= [85 86 86]  var= [2 2 2]',
       'X 1  alpha 3  Beta 6  average= [84 84 84]  max= [85 85 85]  var= [0 0 0]',
       'X 2  alpha 1  Beta 1  average= [86 86 86]  max= [87 87 87]  var= [0 0 0]',
       'X 2  alpha 1  Beta 2  average= [88 88 88]  max= [89 89 89]  var= [0 0 0]',
       'X 2  alpha 1  Beta 6  average= [85 85 85]  max= [88 88 87]  var= [5 5 

In [32]:
mx_wndws=int(np.max(cls_wndws)+1)   # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max =', mx_wndws)
rpt_cnn_clsfctn=3                 #تعداد اجرای شبکه عصبی برای میانگین گیری هر حالت 
aug_amnt_Ttl=np.empty((0,7), int) 
rslts=np.array([])
for X in range(4,6):                               #داده افزایی تا چند برابر کلاس اکثریت
    mx_wndws=int(np.max(cls_wndws))   # حداکثر تعداد پنجره ی موجود بین کلاس ها
    mx_wndws=int(X*mx_wndws)          #
    for alpha in range(1,4):
        rprt=np.array([])
        for beta in range(1,4):                        #چند برابر کردن داده افزایی هر کلاس متناسب ضعیف بودن آن کلاس
            if beta==3:
                beta=beta*2
            scors0=np.array([64,78,55,70,94,85,84,85,78,88,63,90])         #Class Recalls without Augmentation
            AgScr=1+alpha*((100-scors0)/(np.max(100-scors0)))**beta        #Class Specific

            f_scr=np.array([83,88,87,83,85,85,85,85])                      #UnAg,Scl.2,Mag.05,Tm.2,Gs1,frqnc2,GAN,LSTMexpgdo
            f_scr_difrnc=f_scr-f_scr[0]
            for z in range (len(f_scr_difrnc)):
                if f_scr_difrnc[z]<0:
                    f_scr_difrnc[z]=0
            f_scr_difrnc=f_scr_difrnc**beta
            f_scr_rtio=f_scr_difrnc/(sum(f_scr_difrnc))
            aug_amnt=np.empty((0,len(f_scr_rtio)-1), int) 

            for i in range (1,cls_num+1):
                aug_amnt=np.append(aug_amnt,[f_scr_rtio[1:]*(mx_wndws*AgScr[i-1]-cls_wndws[i-1])],axis=0)
                #vlum_win=mx_wndws-cls_wndws[i-1]                        #میزان داده افزایی در روش های عمیق (پنجره)
            aug_amnt=np.int16(np.round(aug_amnt))
            aug_amnt_Ttl=np.append(aug_amnt_Ttl,aug_amnt,axis=0)
            #print(np.int16(100*f_scr_rtio))
            #print(aug_amnt)
            rprt=np.empty((0,3),float)
            print(aug_amnt)
            for repeat in range(1,rpt_cnn_clsfctn+1):
                xtrain=np.empty((0,smpl_rte+1), float) 
                for i in range (1,cls_num+1):                            #تعریف آرایه ی پنجره های کلاس ها
                    xtrain=np.append(xtrain,vars()['wndws'+str(i)],axis=0)
                    if aug_amnt[i-1,0]>0:
                        methd=10
                        sort=-3  #-5=Dis_sam_Cls=qlty1 #-4=var_Dis=qlty2 #-3=DisSm/DisOthr=qlty3  #-2=invrs_GDO_papr=qlty4   #-1=GDO_papr=qlty5
                        dp_slct_dstrb=4            #1-sequencial 2-linear 3-beta 4-exponential 5-unfrm
                        Hrzntl=srtd_dataH(methd,i,aug_amnt[i-1,0],dp_slct_dstrb,sort)
                        Vrtcl=srtd_dataV(methd,i,aug_amnt[i-1,0],dp_slct_dstrb,sort)
                        HrzntlVrtcl=np.concatenate((Hrzntl[:,:-1],Vrtcl),axis=1)
                        xtrain=np.append(xtrain,HrzntlVrtcl,axis=0)
                    if aug_amnt[i-1,1]>0:
                        methd=20
                        sort=-3
                        dp_slct_dstrb=2
                        Hrzntl=srtd_dataH(methd,i,aug_amnt[i-1,1],dp_slct_dstrb,sort)
                        Vrtcl=srtd_dataV(methd,i,aug_amnt[i-1,1],dp_slct_dstrb,sort)
                        HrzntlVrtcl=np.concatenate((Hrzntl[:,:-1],Vrtcl),axis=1)
                        xtrain=np.append(xtrain,HrzntlVrtcl,axis=0)
                    if aug_amnt[i-1,2]>0:
                        methd=30
                        sort=-3
                        dp_slct_dstrb=4
                        Hrzntl=srtd_dataH(methd,i,aug_amnt[i-1,2],dp_slct_dstrb,sort)
                        Vrtcl=srtd_dataV(methd,i,aug_amnt[i-1,2],dp_slct_dstrb,sort)
                        HrzntlVrtcl=np.concatenate((Hrzntl[:,:-1],Vrtcl),axis=1)
                        xtrain=np.append(xtrain,HrzntlVrtcl,axis=0)
                    if aug_amnt[i-1,3]>0:
                        methd=40
                        sort=-1
                        dp_slct_dstrb=4
                        Hrzntl=srtd_dataH(methd,i,aug_amnt[i-1,3],dp_slct_dstrb,sort)
                        Vrtcl=srtd_dataV(methd,i,aug_amnt[i-1,3],dp_slct_dstrb,sort)
                        HrzntlVrtcl=np.concatenate((Hrzntl[:,:-1],Vrtcl),axis=1)
                        xtrain=np.append(xtrain,HrzntlVrtcl,axis=0)
                    if aug_amnt[i-1,4]>0:
                        methd=50
                        sort=-3
                        dp_slct_dstrb=4
                        Hrzntl=srtd_dataH(methd,i,aug_amnt[i-1,4],dp_slct_dstrb,sort)
                        Vrtcl=srtd_dataV(methd,i,aug_amnt[i-1,4],dp_slct_dstrb,sort)
                        HrzntlVrtcl=np.concatenate((Hrzntl[:,:-1],Vrtcl),axis=1)
                        xtrain=np.append(xtrain,HrzntlVrtcl,axis=0)
                    if aug_amnt[i-1,5]>0:
                        methd=60
                        sort=-1
                        dp_slct_dstrb=4
                        Hrzntl=srtd_dataH(methd,i,aug_amnt[i-1,5],dp_slct_dstrb,sort)
                        Vrtcl=srtd_dataV(methd,i,aug_amnt[i-1,5],dp_slct_dstrb,sort)
                        HrzntlVrtcl=np.concatenate((Hrzntl[:,:-1],Vrtcl),axis=1)
                        xtrain=np.append(xtrain,HrzntlVrtcl,axis=0)
                    if aug_amnt[i-1,6]>0:
                        methd=70
                        sort=-1
                        dp_slct_dstrb=2
                        Hrzntl=srtd_dataH(methd,i,aug_amnt[i-1,6],dp_slct_dstrb,sort)
                        Vrtcl=srtd_dataV(methd,i,aug_amnt[i-1,6],dp_slct_dstrb,sort)
                        HrzntlVrtcl=np.concatenate((Hrzntl[:,:-1],Vrtcl),axis=1)
                        xtrain=np.append(xtrain,HrzntlVrtcl,axis=0)
                    #print('class ', i, 'train shape = ', np.shape(xtrain))

                # Normalization train windows
                mx_aug=np.max(xtrain[:,:-1])
                mx=np.max(ecg[:,:-1])
                mn_aug=np.min(xtrain[:,:-1])
                mn=np.min(ecg[:,:-1])
                xtrain[:,:-1]= 2*(xtrain[:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1


                ###Test Windows:
                wndws_test=np.empty((0,len(wndws1[0])), float)
                i=0
                windws=np.array([])
                for cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های آموزش اصلی
                    vars()['wndws_tst'+str(cls)]=np.empty((0,len(wndws1[0])), float)
                    vars()['wndws_tst'+str(cls)]=np.append(vars()['wndws_tst'+str(cls)],vars()['ecg_tst'+str(cls)],axis=0)   

                # Normalization test windows
                for i in range (1,cls_num+1):                
                    cls=i
                    #print('\n cls', i, ' >> ')
                    #print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
                    #print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
                    vars()['wndws_tst'+str(i)][:,:-1]= 2*(vars()['wndws_tst'+str(i)][:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1
                    #print('after normalizing >>')
                    #print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
                    #print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
                #print("each class and its windows =")                    #تجمیع کلاس های تست
                for cls in range (1,cls_num+1):                                # آرایه ی تعداد پنجره ی هر کلاس
                    wns=len(vars()['wndws_tst'+str(cls)])
                    wndws_test=np.append(wndws_test,vars()['wndws_tst'+str(cls)],axis=0)
                Xtest=wndws_test[:,:-1]
                ytest=np.int16(wndws_test[:,-1])

                trainx=np.random.permutation(xtrain)
                Xtrain=np.array(trainx[:,:-1])
                ytrain=np.int16(trainx[:,-1])

                X_train=np.array(Xtrain)
                y_train=np.array(ytrain)
                X_test=np.array(Xtest)
                y_test=np.array(ytest)
                X_valid=np.array(Xtrain)
                y_valid=np.array(ytrain)
                #print('X_train => ', X_train.shape)
                #print('y_train => ', y_train.shape)
                #print('X_test  => ', X_test.shape)
                #print('y_test  => ', y_test.shape)
                #print('X_valid  => ', X_valid.shape)
                #print('y_valid  => ', y_valid.shape)

                y_train_cat=to_categorical(y_train)
                y_valid_cat=to_categorical(y_valid)
                y_test_cat=to_categorical(y_test)
                X_train = np.expand_dims(X_train, axis=2)
                X_valid = np.expand_dims(X_valid, axis=2)
                X_test = np.expand_dims(X_test, axis=2)

                ####### NETWORK #######
                accuracy=0
                acc_crs=np.array([])
                ##for i in range (10):
                ##    if accuracy<0.2 :
                verbose, epochs, batch_size = 0, 1000, btch
                n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
                steps_per_epoch = len(X_train)//batch_size
                validation_steps = len(X_valid)//batch_size # if you have test data
                model_crs = Sequential()
                BatchNormalization()
                model_crs.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
                #model_crs.add(MaxPooling1D(pool_size=3))
                BatchNormalization()
                model_crs.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
                #model_crs.add(Dropout(0.5))
                model_crs.add(MaxPooling1D(pool_size=3))
                BatchNormalization()
                model_crs.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
                model_crs.add(MaxPooling1D(pool_size=3))
                BatchNormalization()
                model_crs.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
                model_crs.add(MaxPooling1D(pool_size=3))
                BatchNormalization()
                model_crs.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
                model_crs.add(MaxPooling1D(pool_size=3))
                '''model_crs.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
                model_crs.add(MaxPooling1D(pool_size=3))
                model_crs.add(Conv1D(filters=1024, kernel_size=3, activation='relu'))
                model_crs.add(MaxPooling1D(pool_size=3))
                '''
                model_crs.add(Flatten())
                #model_crs.add(Dropout(0.5))
                model_crs.add(Dense(2000, activation='relu'))
                model_crs.add(Dense(n_outputs, activation='softmax'))
                model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
                # fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy
                earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 20, restore_best_weights = True)
                model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])
                # evaluate model_crs
                _, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=verbose)
                predict_x=model_crs.predict(X_test)              # Function 1
                y_pred_crs=maxindx(predict_x)                    # function from augment.py to remove 0 index predictions
                rprt0=classification_report(y_test, y_pred_crs,output_dict=True)
                rprt_row=np.array([])
                rprt_row=np.append(rprt_row,rprt0['accuracy'])
                rprt_row=np.append(rprt_row,rprt0['macro avg']['f1-score'])
                rprt_row=np.append(rprt_row,rprt0['weighted avg']['f1-score'])
                rprt=np.append(rprt,[rprt_row],axis=0)
            #print('rprt=',rprt)
            maxm=rprt[np.argmax(rprt[:,1])]
            #print('maxm=',maxm)
            avrg=np.mean(rprt,axis=0)
            #print('avrg=',avrg)
            rslts=np.append(rslts,['X '+str(X)+'  alpha '+str(alpha) +'  Beta '+str(beta)+'  average= '+str(np.int16(100*avrg))+'  max= '+str(np.int16(100*maxm))+'  var= '+str(np.int16(np.var(100*rprt,axis=0)))],axis=0)
            print('rslts=',rslts[-1])


 max = 32
[[57 45  0 23 23 23 23]
 [45 36  0 18 18 18 18]
 [64 51  0 26 26 26 26]
 [52 42  0 21 21 21 21]
 [33 26  0 13 13 13 13]
 [40 32  0 16 16 16 16]
 [40 32  0 16 16 16 16]
 [40 32  0 16 16 16 16]
 [45 36  0 18 18 18 18]
 [37 30  0 15 15 15 15]
 [58 46  0 23 23 23 23]
 [36 29  0 14 14 14 14]]
12/12 [==============================] - 0s 3ms/step
rslts= X 4  alpha 1  Beta 1  average= [86 86 86]  max= [87 87 87]  var= [0 0 0]
[[76 49  0 12 12 12 12]
 [54 35  0  9  9  9  9]
 [96 61  0 15 15 15 15]
 [65 42  0 10 10 10 10]
 [42 27  0  7  7  7  7]
 [47 30  0  7  7  7  7]
 [48 31  0  8  8  8  8]
 [47 30  0  8  8  8  8]
 [54 35  0  9  9  9  9]
 [45 29  0  7  7  7  7]
 [78 50  0 12 12 12 12]
 [44 28  0  7  7  7  7]]
12/12 [==============================] - 0s 3ms/step
rslts= X 4  alpha 1  Beta 2  average= [88 87 87]  max= [88 88 88]  var= [0 0 0]
[[ 99  26   0   0   0   0   0]
 [ 75  20   0   0   0   0   0]
 [171  45   0   1   1   1   1]
 [ 82  22   0   0   0   0   0]
 [ 74  19   0   0   0

12/12 [==============================] - 0s 3ms/step
rslts= X 5  alpha 2  Beta 2  average= [87 87 87]  max= [87 87 87]  var= [0 0 0]
[[161  42   0   1   1   1   1]
 [101  26   0   0   0   0   0]
 [340  89   0   1   1   1   1]
 [119  31   0   0   0   0   0]
 [ 98  26   0   0   0   0   0]
 [ 97  26   0   0   0   0   0]
 [ 97  26   0   0   0   0   0]
 [ 98  26   0   0   0   0   0]
 [101  26   0   0   0   0   0]
 [ 98  26   0   0   0   0   0]
 [173  45   0   1   1   1   1]
 [ 98  26   0   0   0   0   0]]
12/12 [==============================] - 0s 3ms/step
rslts= X 5  alpha 2  Beta 6  average= [87 86 86]  max= [89 88 88]  var= [1 2 2]
[[146 117   0  58  58  58  58]
 [104  83   0  41  41  41  41]
 [174 139   0  69  69  69  69]
 [128 102   0  51  51  51  51]
 [ 55  44   0  22  22  22  22]
 [ 82  66   0  33  33  33  33]
 [ 85  68   0  34  34  34  34]
 [ 82  66   0  33  33  33  33]
 [104  83   0  41  41  41  41]
 [ 73  59   0  29  29  29  29]
 [149 119   0  60  60  60  60]
 [ 67  54   0  27  2

In [33]:
rslts

array(['X 4  alpha 1  Beta 1  average= [86 86 86]  max= [87 87 87]  var= [0 0 0]',
       'X 4  alpha 1  Beta 2  average= [88 87 87]  max= [88 88 88]  var= [0 0 0]',
       'X 4  alpha 1  Beta 6  average= [88 88 88]  max= [90 89 89]  var= [4 5 5]',
       'X 4  alpha 2  Beta 1  average= [87 87 87]  max= [88 88 88]  var= [1 1 1]',
       'X 4  alpha 2  Beta 2  average= [90 90 90]  max= [91 91 91]  var= [0 0 0]',
       'X 4  alpha 2  Beta 6  average= [87 87 87]  max= [88 88 88]  var= [1 0 0]',
       'X 4  alpha 3  Beta 1  average= [87 87 87]  max= [88 88 88]  var= [1 1 1]',
       'X 4  alpha 3  Beta 2  average= [89 89 89]  max= [89 89 89]  var= [0 0 0]',
       'X 4  alpha 3  Beta 6  average= [86 86 86]  max= [90 89 89]  var= [5 6 6]',
       'X 5  alpha 1  Beta 1  average= [87 87 87]  max= [89 89 89]  var= [7 7 7]',
       'X 5  alpha 1  Beta 2  average= [87 86 86]  max= [89 89 89]  var= [7 8 8]',
       'X 5  alpha 1  Beta 6  average= [87 87 86]  max= [88 88 88]  var= [1 1 1]',
    